In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()




from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()
print(len(train_cands),len(dev_cands))

8272 888


In [2]:
# from load_external_annotations import load_external_labels

# load_external_labels(session, ChemicalDisease, split=2, annotator='gold')

from snorkel.annotations import load_gold_labels
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]

gold_labels_test = [L[0,0] for L in L_gold_test]

from snorkel.learning.utils import MentionScorer

import numpy as np
unique, counts = np.unique(gold_labels_dev, return_counts=True)
print(dict(zip(unique, counts)))


unique, counts = np.unique(gold_labels_test, return_counts=True)
print(dict(zip(unique, counts)))

print(len(gold_labels_dev),len(gold_labels_test))

Instructions for updating:
Use the retry module or similar alternatives.
{0: 592, 1: 296}
{0: 3115, 1: 1505}
888 4620


In [3]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [4]:
import re
from snorkel.lf_helpers import (
    get_tagged_text,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

In [5]:
import bz2
from six.moves.cPickle import load

with bz2.BZ2File('data/ctd.pkl.bz2', 'rb') as ctd_f:
    ctd_unspecified, ctd_therapy, ctd_marker = load(ctd_f)

In [1]:
# causal_past = ['induced', 'caused', 'due']

#drop 2
causal_past = ['induced','caused']

# treat = ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap']

# drop 2
treat = ['effective', 'slow', 'promise', ]


# procedure, following = ['inject', 'administrat'], ['following']

#drop 2
procedure, following = ['administrat'], ['following']

# uncertain = ['combin', 'possible', 'unlikely']


# drop 2
uncertain = ['combin', 'possible']

In [12]:
##### Discrete #########

def cand_in_ctd_unspecified(c):
    return 1 if c.get_cids() in ctd_unspecified else 0

def cand_in_ctd_therapy(c):
    return 1 if c.get_cids() in ctd_therapy else 0

def cand_in_ctd_marker(c):
    return 1 if c.get_cids() in ctd_marker else 0


def LF_in_ctd_unspecified(c):
    if(cand_in_ctd_unspecified(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_therapy(c):
    if(cand_in_ctd_therapy(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_marker(c):
    if(cand_in_ctd_marker(c)==1):
        return (1,1)
    else:
        return (0,0)



# List to parenthetical
def ltp(x):
    return '(' + '|'.join(x) + ')'

def LF_induce(c):
    return (1,1) if re.search(r'{{A}}.{0,20}induc.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


def LF_d_induced_by_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + '.{0,9}(by|to).{0,50}', 1)==1):
        return (1,1)
    return (0,0)

def LF_d_induced_by_c_tight(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + ' (by|to) ', 1)==1):
        return (1,1)
    return (0,0)

def LF_induce_name(c):
    return (1,1) if 'induc' in c.chemical.get_span().lower() else (0,0)     

causal = ['cause[sd]?', 'induce[sd]?', 'associated with']
def LF_c_cause_d(c):
    return (1,1) if (
        re.search(r'{{A}}.{0,50} ' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
        and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
    ) else (0,0)


def LF_d_treat_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d(c):
    if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_treat_d(c):
    if (rule_regex_search_before_B(c, ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d_wide(c):
    if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_d(c):
    return (1,1) if ('{{A}} {{B}}' in get_tagged_text(c)) else (0,0)

def LF_c_induced_d(c):
    return (1,1) if (
        ('{{A}} {{B}}' in get_tagged_text(c)) and 
        (('-induc' in c[0].get_span().lower()) or ('-assoc' in c[0].get_span().lower()))
        ) else (0,0)

def LF_improve_before_disease(c):
    if(rule_regex_search_before_B(c, 'improv.*', -1)==-1):
        return (-1,1)
    return (0,0)

pat_terms = ['in a patient with ', 'in patients with']
def LF_in_patient_with(c):
    return (-1,1) if re.search(ltp(pat_terms) + '{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


def LF_uncertain(c):
    if (rule_regex_search_before_A(c, ltp(uncertain) + '.*', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_induced_other(c):
    if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
        return (-1,1)
    return (0,0)  

def LF_far_c_d(c):
    if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_far_d_c(c):
    if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_risk_d(c):
    if (rule_regex_search_before_B(c, 'risk of ', 1)==1):
        return (1,1)
    return (0,0)

def LF_develop_d_following_c(c):
    return (1,1) if re.search(r'develop.{0,25}{{B}}.{0,25}following.{0,25}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


def LF_d_following_c(c):
    return (1,1) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)

def LF_measure(c):
    return (-1,1) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)

def LF_level(c):
    return (-1,1) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)

def LF_neg_d(c):
    return (-1,1) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
                'seems', 'suggests', 'risk', 'implicated',
               'the aim', 'to (investigate|assess|study)']

WEAK_RGX = r'|'.join(WEAK_PHRASES)

def LF_weak_assertions(c):
    return (-1,1) if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else (0,0)


def LF_ctd_marker_c_d(c):
    l,s = LF_c_d(c)
    cl = cand_in_ctd_marker(c)
    return (l*cl,s*cl)

def LF_ctd_marker_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_marker(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)

def LF_ctd_therapy_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_therapy(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_unspecified(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_unspecified(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)


# def LF_ctd_marker_c_d(c):
#     return LF_c_d(c) * cand_in_ctd_marker(c)

# def LF_ctd_marker_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_marker(c)

# def LF_ctd_therapy_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_therapy(c)

# def LF_ctd_unspecified_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_unspecified(c)

# def LF_ctd_unspecified_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_unspecified(c)

def LF_closer_chem(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical closer than @dist/2 in either direction
    sent = c.get_parent()
    closest_other_chem = float('inf')
    for i in range(dis_end, min(len(sent.words), dis_end + dist // 2)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    for i in range(max(0, dis_start - dist // 2), dis_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    return (0,0)

def LF_closer_dis(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical disease than @dist/8 in either direction
    sent = c.get_parent()
    for i in range(chem_end, min(len(sent.words), chem_end + dist // 8)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    for i in range(max(0, chem_start - dist // 8), chem_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    return (0,0)


LFs = [LF_c_cause_d,LF_c_d,LF_c_induced_d,LF_c_treat_d,LF_c_treat_d_wide,LF_closer_chem,
    LF_closer_dis,LF_ctd_marker_c_d,LF_ctd_marker_induce,LF_ctd_therapy_treat,
    LF_ctd_unspecified_treat,LF_ctd_unspecified_induce,LF_d_following_c,
    LF_d_induced_by_c,LF_d_induced_by_c_tight,LF_d_treat_c,LF_develop_d_following_c,
    LF_far_c_d,LF_far_d_c,LF_improve_before_disease,LF_in_ctd_therapy,
    LF_in_ctd_marker,LF_in_patient_with,LF_induce,LF_induce_name,LF_induced_other,
    LF_level,LF_measure,LF_neg_d,LF_risk_d,LF_treat_d,LF_uncertain,LF_weak_assertions
]

LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]
print(len(LFs),len(LF_l))

33 33


In [13]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_discrete_drop1",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_discrete_drop1",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_discrete_drop1",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 28-6-2018, 11:52:26
500data points labelled in 0.0042977412541707356 mins
500data points labelled in 0.011832578976949056 mins
1000data points labelled in 0.01585382620493571 mins
1500data points labelled in 0.0200410525004069 mins
2000data points labelled in 0.024696004390716553 mins
2500data points labelled in 0.02877231438954671 mins
3000data points labelled in 0.0332605520884196 mins
3500data points labelled in 0.03739567995071411 mins
4000data points labelled in 0.041582767168680826 mins
4500data points labelled in 0.04613828659057617 mins
5000data points labelled in 0.05049457550048828 mins
5500data points labelled in 0.05473352273305257 mins
6000data points labelled in 0.05886332591374715 mins
6500data points labelled in 0.06321536699930827 mins
7000data points labelled in 0.06762253443400065 mins
7500data points labelled in 0.07199596166610718 mins
8000data points labelled in 0.07666500806808471 mins
500data points labelled in 0.08407185077667237 mins
1000data point

In [5]:
# import numpy as np

# def distanceCD(c):
#     dist = 0
#     chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
#     dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
#     if dis_start < chem_start:
#         dist = chem_start - dis_end
#     else:
#         dist = dis_start - chem_end
#     return dist/5000


# def distanceCD_(c,l):
#     dist = []
#     for w in l:
#         pattern = r'({{A}})(.*)('+w+r')(.*)({{B}})'
#         matchObj = re.search(pattern, get_tagged_text(c), flags=re.I)
#         if(matchObj):
#             match_groups = matchObj.group(2,4)
#             dist.append(sum([len(mg) for mg in match_groups]))
#     if(len(dist)>0):
#         return min(dist)
#     return 0

# def distanceDC_(c,l):
#     dist = []
#     for w in l:
#         pattern = r'({{B}})(.*)('+w+r')(.*)({{A}})'
#         matchObj = re.search(pattern, get_tagged_text(c), flags=re.I)
#         if(matchObj):
#             match_groups = matchObj.group(2,4)
#             dist.append(sum([len(mg) for mg in match_groups]))
#     if(len(dist)>0):
#         return min(dist)
#     return 0

   

# def levenshtein(source, target):
#     if len(source) < len(target):
#         return levenshtein(target, source)

#     # So now we have len(source) >= len(target).
#     if len(target) == 0:
#         return len(source)

#     # We call tuple() to force strings to be used as sequences
#     # ('c', 'a', 't', 's') - numpy uses them as values by default.
#     source = np.array(tuple(source))
#     target = np.array(tuple(target))

#     # We use a dynamic programming algorithm, but with the
#     # added optimization that we only need the last two rows
#     # of the matrix.
#     previous_row = np.arange(target.size + 1)
#     for s in source:
#         # Insertion (target grows longer than source):
#         current_row = previous_row + 1

#         # Substitution or matching:
#         # Target and source items are aligned, and either
#         # are different (cost of 1), or are the same (cost of 0).
#         current_row[1:] = np.minimum(
#                 current_row[1:],
#                 np.add(previous_row[:-1], target != s))

#         # Deletion (target grows shorter than source):
#         current_row[1:] = np.minimum(
#                 current_row[1:],
#                 current_row[0:-1] + 1)

#         previous_row = current_row

#     return previous_row[-1]

In [15]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    write_to_file(wordvec_unavailable) 
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [19]:
##### Smooth LFs #########

def cand_in_ctd_unspecified(c):
    return 1 if c.get_cids() in ctd_unspecified else 0

def cand_in_ctd_therapy(c):
    return 1 if c.get_cids() in ctd_therapy else 0

def cand_in_ctd_marker(c):
    return 1 if c.get_cids() in ctd_marker else 0


def LF_in_ctd_unspecified(c):
    if(cand_in_ctd_unspecified(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_therapy(c):
    if(cand_in_ctd_therapy(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_marker(c):
    if(cand_in_ctd_marker(c)==1):
        return (1,1)
    else:
        return (0,0)


# List to parenthetical
def ltp(x):
    return '(' + '|'.join(x) + ')'

def LF_induce(c):
    return (1,1) if re.search(r'{{A}}.{0,20}induc.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

# def LF_induce(c):
#     return (1,distanceCD_(c,['induc'])) if re.search(r'{{A}}.*induc.*{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


# def LF_d_induced_by_c(c):
#     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + '.{0,9}(by|to).{0,50}', 1)==1):
#         return (1,1)
#     return (0,0)

def LF_d_induced_by_c(c):
    sc = 0
    word_vectors = get_word_vectors(get_between_tokens(c))
    for w in causal_past:
        sc=max(sc,get_similarity(word_vectors,w))
    return (1,sc)

def LF_d_induced_by_c_tight(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + ' (by|to) ', 1)==1):
        return (1,1)
    return (0,0)

# def LF_d_induced_by_c_tight(c):
#     if (rule_regex_search_btw_BA(c, '.*' + ltp(causal_past) + ' (by|to) ', 1)==1):
#         return (1,(1-distanceDC_(c,causal_past)))
#     return (0,0)

def LF_induce_name(c):
    return (1,1) if 'induc' in c.chemical.get_span().lower() else (0,0)     

causal = ['cause[sd]?', 'induce[sd]?', 'associated with']
def LF_c_cause_d(c):
    return (1,1) if (
        re.search(r'{{A}}.{0,50} ' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
        and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
    ) else (0,0)


# def LF_c_cause_d(c):
#     return (1,(1-distanceCD_(c,causal))) if (
#         re.search(r'{{A}}.* ' + ltp(causal) + '.*{{B}}', get_tagged_text(c), re.I)
#         and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
#     ) else (0,0)


def LF_d_treat_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

# def LF_d_treat_c(c):
#     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
#         return (-1,1-distanceDC_(c,treat))
#     return (0,0)


def LF_c_treat_d(c):
    if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

# def LF_c_treat_d(c):
#     if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
#         return (-1,1-distanceCD_(c,treat))
#     return (0,0)

# def LF_treat_d(c):
#     if (rule_regex_search_before_B(c, ltp(treat) + '.{0,50}', -1)==-1):
#         return (-1,1)
#     return (0,0)

def LF_treat_d(c):
    sc = 0
    word_vectors = get_word_vectors(get_left_tokens(c[1],7))
    for w in treat:
        sc=max(sc,get_similarity(word_vectors,w))
    if(re.search('(not|no|none) .* {{B}}', get_tagged_text(c), re.I)):
        return (0,0)
    else:
        return (-1,sc)

def LF_c_treat_d_wide(c):
    if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
        return (-1,1)
    return (0,0)

# def LF_c_treat_d_wide(c):
#     if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
#         return (-1,1-distanceCD_(c,treat))
#     return (0,0)

def LF_c_d(c):
    return (1,1) if ('{{A}} {{B}}' in get_tagged_text(c)) else (0,0)

def LF_c_induced_d(c):
    return (1,1) if (
        ('{{A}} {{B}}' in get_tagged_text(c)) and 
        (('-induc' in c[0].get_span().lower()) or ('-assoc' in c[0].get_span().lower()))
        ) else (0,0)

def LF_improve_before_disease(c):
    if(rule_regex_search_before_B(c, 'improv.*', -1)==-1):
        return (-1,1)
    return (0,0)


# def distanceImproveBeforeDisease(c):
#     m=re.search(r'(improv)(.*)({{B}})', get_tagged_text(c), flags=re.I)
#     if(m):
#         return len(m.group(2))/5000
#     return 0


# def LF_improve_before_disease(c):
#     if(rule_regex_search_before_B(c, 'improv.*', -1) == -1):
#         return (-1,1-distanceImproveBeforeDisease(c))
#     else:
#         return (0,0)


pat_terms = ['in a patient with ', 'in patients with']
def LF_in_patient_with(c):
    return (-1,1) if re.search(ltp(pat_terms) + '{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


# def LF_uncertain(c):
#     if (rule_regex_search_before_A(c, ltp(uncertain) + '.*', -1)==-1):
#         return (-1,1)
#     return (0,0)

def LF_uncertain(c):
    sc = 0
    word_vectors = get_word_vectors(get_left_tokens(c[1],7))
    for w in uncertain:
        sc=max(sc,get_similarity(word_vectors,w))
    if(re.search('(not|no|none) .* {{B}}', get_tagged_text(c), re.I)):
        return (0,0)
    else:
        return (-1,sc)

def LF_induced_other(c):
    if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
        return (-1,1)
    return (0,0)  

# def LF_induced_other(c):
#     if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)  

def LF_far_c_d(c):
    if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

# def LF_far_c_d(c):
#     if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)

def LF_far_d_c(c):
    if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

# def LF_far_d_c(c):
#     if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)


# def LF_risk_d(c):
#     if (rule_regex_search_before_B(c, 'risk of ', 1)==1):
#         return (1,1)
#     return (0,0)


def LF_risk_d(c):
    sc = 0
    word_vectors = get_word_vectors(get_left_tokens(c[1],7))
    sc=max(sc,get_similarity(word_vectors,'risk'))
    return (1,sc)


def LF_develop_d_following_c(c):
    return (1,1) if re.search(r'develop.{0,25}{{B}}.{0,25}following.{0,25}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceDevFol(c):
#     dist = 0
#     matchObj = re.search(r'(develop)(.*)({{B}})(.*)(following)(.*)({{A}})', get_tagged_text(c), flags=re.I)
#     if(matchObj):
#         match_groups = matchObj.group(2,4,6)
#         dist = sum([len(mg) for mg in match_groups])
#     return dist/5000

# def LF_develop_d_following_c(c):
#     return (1,1-distanceDevFol(c)) if re.search(r'develop.*{{B}}.*following.*{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# def LF_d_following_c(c):
#     return (1,distanceDFollC(c)) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)


# def LF_d_following_c(c):
#     return (1,1-distanceDC_(c,following)) if re.search('{{B}}.*' + ltp(following) + '.*{{A}}.*' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)


def LF_d_following_c(c):
    return (1,1) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)

def LF_measure(c):
    return (-1,1) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceMeasureA(c):
#     m = re.search('(measur)(.*)({{A}})', get_tagged_text(c), flags=re.I) 
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0

# def LF_measure(c):
#     return (-1,distanceMeasureA(c)) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


def LF_level(c):
    return (-1,1) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceLevel(c):
#     m = re.search('({{A}})(.*)(level)', get_tagged_text(c), flags=re.I)
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0

# def LF_level(c):
#     return (-1,distanceLevel(c)) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)

def LF_neg_d(c):
    return (-1,1) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceNeg(c):
#     m = re.search('(none|not|no)(.*)({{B}})', get_tagged_text(c), flags=re.I)
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0


# def LF_neg_d(c):
#     return (-1,distanceNeg(c)) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
                'seems', 'suggests', 'risk', 'implicated',
               'the aim', 'to (investigate|assess|study)']

WEAK_RGX = r'|'.join(WEAK_PHRASES)

def LF_weak_assertions(c):
    return (-1,1) if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else (0,0)


def LF_ctd_marker_c_d(c):
    l,s = LF_c_d(c)
    cl = cand_in_ctd_marker(c)
    return (l*cl,s*cl)

def LF_ctd_marker_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_marker(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)

def LF_ctd_therapy_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_therapy(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_unspecified(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_unspecified(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)


# def LF_ctd_marker_c_d(c):
#     return LF_c_d(c) * cand_in_ctd_marker(c)

# def LF_ctd_marker_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_marker(c)

# def LF_ctd_therapy_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_therapy(c)

# def LF_ctd_unspecified_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_unspecified(c)

# def LF_ctd_unspecified_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_unspecified(c)

def LF_closer_chem(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical closer than @dist/2 in either direction
    sent = c.get_parent()
    closest_other_chem = float('inf')
    for i in range(dis_end, min(len(sent.words), dis_end + dist // 2)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    for i in range(max(0, dis_start - dist // 2), dis_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    return (0,0)

def LF_closer_dis(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical disease than @dist/8 in either direction
    sent = c.get_parent()
    for i in range(chem_end, min(len(sent.words), chem_end + dist // 8)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    for i in range(max(0, chem_start - dist // 8), chem_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    return (0,0)


LFs = [LF_c_cause_d,LF_c_d,LF_c_induced_d,LF_c_treat_d,LF_c_treat_d_wide,LF_closer_chem,
    LF_closer_dis,LF_ctd_marker_c_d,LF_ctd_marker_induce,LF_ctd_therapy_treat,
    LF_ctd_unspecified_treat,LF_ctd_unspecified_induce,LF_d_following_c,
    LF_d_induced_by_c,LF_d_induced_by_c_tight,LF_d_treat_c,LF_develop_d_following_c,
    LF_far_c_d,LF_far_d_c,LF_improve_before_disease,LF_in_ctd_therapy,
    LF_in_ctd_marker,LF_in_patient_with,LF_induce,LF_induce_name,LF_induced_other,
    LF_level,LF_measure,LF_neg_d,LF_risk_d,LF_treat_d,LF_uncertain,LF_weak_assertions
]

LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]
print(len(LFs),len(LF_l))

33 33


In [20]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_smooth_drop1",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_smooth_drop1",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_smooth_drop1",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 28-6-2018, 12:6:6
500data points labelled in 0.14652823607126872 mins
500data points labelled in 0.40101074377695717 mins
1000data points labelled in 0.5394020040829977 mins
1500data points labelled in 0.6851861039797466 mins
2000data points labelled in 0.8272844990094502 mins
2500data points labelled in 0.9598392327626546 mins
3000data points labelled in 1.1012714584668477 mins
3500data points labelled in 1.2373753627141317 mins
4000data points labelled in 1.3720657348632812 mins
4500data points labelled in 1.5356725136439004 mins
5000data points labelled in 1.6798930168151855 mins
5500data points labelled in 1.8252692341804504 mins
6000data points labelled in 1.9635700742403666 mins
6500data points labelled in 2.1223971883455914 mins
7000data points labelled in 2.283395326137543 mins
7500data points labelled in 2.4368956128756207 mins
8000data points labelled in 2.592478601137797 mins
500data points labelled in 2.8313360015551248 mins
1000data points labelled in 2.9753644

In [21]:
import pandas as pd
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import average_precision_score

In [22]:
LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]

In [23]:
dev_L_S = np.load("dev_L_S_discrete_drop1.npy")
test_L_S = np.load("test_L_S_discrete_drop1.npy")
train_L_S = np.load("train_L_S_discrete_drop1.npy")
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

(4620, 2, 33) (888, 2, 33) (8272, 2, 33)


In [27]:
import scipy.sparse as sp
import _pickle as pkl


L_train = np.load("train_L_S_discrete_drop1.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)


from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, epochs = 100)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))

(8272, 33)
started at: 28-6-2018, 12:34:33
Inferred cardinality: 2
trained in  0:01:25.926577


In [28]:

L_dev = np.load("dev_L_S_discrete_drop1.npy")
L_dev = L_dev[:,0,:].astype(int)
print(np.array(L_dev).shape)
L_dev = sp.csr_matrix(L_dev)

marginals = gen_model.marginals(L_dev)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_dev),GenLabels,average="binary"))


(888, 33)
(888,)
(888,)
(0.5355191256830601, 0.9932432432432432, 0.6958579881656806, None)


In [29]:

L_test = np.load("test_L_S_discrete_drop1.npy")
L_test = L_test[:,0,:].astype(int)
print(np.array(L_test).shape)
L_test = sp.csr_matrix(L_test)

marginals = gen_model.marginals(L_test)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_test),GenLabels,average="binary"))


(4620, 33)
(4620,)
(4620,)
(0.5334545454545454, 0.9747508305647841, 0.6895417156286721, None)


In [30]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [31]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

33


In [32]:

def train(lr,ep,th,af,batch_size=32,LF_acc=None,LF_rec=None,pcl=np.array([-1,1],dtype=np.float64),norm=True,\
          smooth=True,penalty=0,p3k=3,alp=1,Gamma=1.0,debug=True):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    ## Gamma : penalty tuning parameter
    
    BATCH_SIZE = batch_size
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))
        test_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
        test_init_op = iterator.make_initializer(test_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)
        
        

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    
        g = tf.convert_to_tensor(Gamma, dtype=tf.float64)
        
        if(penalty in [4,5,7,8,9,10,11]):
            LF_a = tf.convert_to_tensor(LF_acc, dtype=tf.float64)
        
        if(penalty in [6,7,11,12]):
            LF_r = tf.convert_to_tensor(LF_rec, dtype=tf.float64)
        
        if(debug):
            print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        if(debug):
            print("s",s)
            print("l",l)
#         print(s.graph)
        if(smooth):
            s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
            if(debug):
                print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            if(debug):
                print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        if(debug):
            print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        if(debug):
            print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        if(debug):
            print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            if(debug):
                print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            if(debug):
                print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")
            
        ### for precision and recall t_pout
        def p_t_pout(j):
            Lj = tf.map_fn(lambda li : tf.gather(li,j),l)
            if(debug):
                print("sft Lj",Lj)
            kj = tf.gather(k,j)
            if(debug):
                print("sft kj",kj)
            indices = tf.where(tf.equal(Lj,kj))
            if(debug):
                print("sft indices",indices)
            li_lij_eq_kj = tf.gather(l,tf.squeeze(indices,1))
            if(smooth):
                si_lij_eq_kj = tf.gather(s_,tf.squeeze(indices,1))
            if(debug):
                print("sft l_ij_eq_kj",li_lij_eq_kj)
            if(smooth):
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                prec_t_pout = (tf.matmul(li_lij_eq_kj*si_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            else:
                prec_t_pout = (tf.matmul(li_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            return prec_t_pout
        
        def r_t_pout(j):
            kj = tf.gather(k,j)
            if(debug):
                print("r_t_pout kj",kj)

            if(smooth):
                rec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                rec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                rec_t_pout = (tf.matmul(l*s_*kj, thetas,transpose_b=True))/rec_z
            else:
                rec_t_pout = (tf.matmul(l*kj, thetas,transpose_b=True))/rec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            return rec_t_pout
            
           
        def softplus_p(j):
            aj = tf.gather(LF_a,j)
            if(debug):
                print("sft aj",aj)
            f_p =  tf.reduce_sum(aj - p_t_pout(j))
            if(debug):
                print("f_p",f_p)
            sft_p = tf.nn.softplus(f_p,name="sft_p")
            if(debug):
                print("sft_p",sft_p)
            return sft_p
        
        def softplus_r(j):
            rj = tf.gather(LF_r,j)
            if(debug):
                print("sft aj",rj)
            f_r =  rj - tf.reduce_sum(r_t_pout(j)) 
            if(debug):
                print("f_r",f_r)
            sft_r = tf.nn.softplus(f_r,name="sft_r")
            if(debug):
                print("sft_r",sft_r)
            return sft_r
            
#         logsft = tf.map_fn(lambda j: tf.log(softplus(j)),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
#         sft  =  tf.map_fn(lambda j: softplus(j),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
        
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        if(debug):
            print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        if(debug):
            print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        if(debug):
            print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +(g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -(g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +(g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 4):
            print("precision penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 5):
            print("precision log(softplus) penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: tf.log(softplus_p(j)),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 6):
            print("recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 7):
            print("precision and recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 8):
            print("precision and sign 1 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 9):
            print("precision and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 10):
            print("precision and sign 3 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 11):
            print("precision and recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 12):
            print("recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        else:
            print("normalized loss")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
       
        if(debug):
            print("loss",loss)
        tf.summary.scalar('loss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        if(debug):
            print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
                        
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)
                print("dev set")
                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_dev,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()
                print("test set")
                sess.run(test_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_test,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(pl),average="binary"))
                print()
                
#             # Initialize an iterator over the validation dataset.
#             sess.run(dev_init_op)
#             a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#             print(a)
#             print(t)

#             unique, counts = np.unique(pl, return_counts=True)
#             print(dict(zip(unique, counts)))

#             print("acc",accuracy_score(true_labels,pl))

# #             predictAndPrint(pl)
#             print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [35]:
train(0.0001,5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False)

k Tensor("Const:0", shape=(33,), dtype=float64)
s Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 207019.3532199147
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 

array([[1],
       [1],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [36]:
import numpy as np
dev_L_S = np.load("dev_L_S_smooth_drop1.npy")
test_L_S = np.load("test_L_S_smooth_drop1.npy")
train_L_S = np.load("train_L_S_smooth_drop1.npy")


print(len(gold_labels_dev),len(gold_labels_test))
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

(4620, 2, 33) (888, 2, 33) (8272, 2, 33)


In [72]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

[2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2
 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2
 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2 2895.2
 2895.2 2895.2 2895.2]
batch-size: 128 alpha-init: 0.6
recall penalty
0 loss 6398141.522691029
dev set
[0.6004729  0.59739159 0.59936347 0.60062471 0.60116366 0.599981
 0.60246111 0.59956937 0.60072455 0.60117509 0.60037405 0.59994626
 0.59907804 0.60006834 0.59968293 0.60120344 0.59775576 0.59947162
 0.60127991 0.59914536 0.59998774 0.59870129 0.6007439  0.60095092
 0.59800611 0.60249813 0.60145662 0.60146048 0.60134149 0.59915694
 0.59898231 0.5991561  0.60017017]
[[1.11810458 0.81495594 1.01027717 1.00055709 1.10679171 1.04200729
  1.16846269 1.0319405  1.14706333 1.06467489 0.98368826 1.06731607
  0.98190972 1.07923261 1.04144353 1.04147307 0.85397106 0.92024795
  1.05943648 0.83563677 1.03594556 0.94213797 0.99543436 1.16334616
  0.87445143 1.17689676 1.06731779 1.0677388

3 loss 6398286.301692839
dev set
[0.69921198 0.69519721 0.69723345 0.69999699 0.69922476 0.69779739
 0.70463791 0.69740182 0.69857676 0.70033934 0.69898583 0.69785257
 0.7023739  0.69829846 0.69811764 0.70343688 0.69681885 0.69766084
 0.7014999  0.70144886 0.69801982 0.69655716 0.70305983 0.7004905
 0.69583635 0.70386856 0.70374368 0.70375765 0.70355299 0.70184583
 0.70111161 0.7012114  0.70023855]
[[1.11921026 0.81715068 1.01241152 1.00218696 1.10874696 1.04413663
  1.16661044 1.03411314 1.14921933 1.0662529  0.98554584 1.06941617
  0.97931776 1.08095913 1.04288074 1.03950785 0.85024054 0.92228974
  1.06089344 0.8333554  1.03795205 0.94428541 0.99312642 1.16346474
  0.87662291 1.17750559 1.06508742 1.06547048 1.05471005 0.98750022
  0.8195482  0.8360818  0.96280235]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

2 loss 6399194.080665224
dev set
[0.89980149 0.8959481  0.89794314 0.89823264 0.89908271 0.89824189
 0.90161925 0.898131   0.89928198 0.89890922 0.89791876 0.89854439
 0.90206433 0.89891647 0.89869463 0.90027115 0.90078861 0.897177
 0.89882721 0.90050445 0.89841519 0.89725859 0.90221831 0.90086592
 0.89656993 0.90004436 0.90242874 0.90271254 0.90125701 0.9019741
 0.89831285 0.89837208 0.89842093]
[[1.11908146 0.81641933 1.01172517 1.00245662 1.10839039 1.04353011
  1.16989706 1.03340145 1.14852913 1.06651726 0.98553215 1.06875
  0.98023669 1.08050189 1.04259793 1.04320056 0.85114926 0.92199773
  1.06148812 0.83449831 1.037343   0.94360251 0.99406175 1.16418069
  0.87591256 1.17936504 1.06696413 1.0667328  1.0575095  0.98845869
  0.82244722 0.83876298 0.96348454]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2975, 1: 1645}
acc 0.7489177489177489
(0.6048632218844985, 0.6611295681063123, 0.6317460317460317, None)

3 

1 loss 3340806.824511354
dev set
[0.60044061 0.59736602 0.59933021 0.60069492 0.60118411 0.59990048
 0.60247536 0.59953988 0.60069447 0.60125008 0.60042628 0.59990467
 0.5990557  0.60002746 0.59965468 0.60121361 0.59774888 0.5995235
 0.60132656 0.59915348 0.59989926 0.59866217 0.60075225 0.60089764
 0.59797018 0.6025313  0.60146786 0.60147205 0.6013575  0.5991374
 0.59899975 0.5991676  0.60027841]
[[1.11817211 0.81498659 1.01031773 1.00051541 1.10689426 1.04205182
  1.1684377  1.03197547 1.14709871 1.06467358 0.98372709 1.06736685
  0.98178406 1.07930423 1.04150313 1.04145167 0.85402554 0.92039422
  1.05936297 0.83562805 1.0360133  0.94218135 0.9954252  1.16347486
  0.87449437 1.1768267  1.06730295 1.06772397 1.05670367 0.98986877
  0.82123577 0.83761575 0.96189112]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

0 loss 3341406.5175731657
dev set
[0.80090646 0.7977553  0.79971414 0.79972182 0.8007267  0.80003677
 0.8020275  0.79992698 0.80107819 0.80040008 0.7995158  0.80028731
 0.80021836 0.80042249 0.80006438 0.80079401 0.79892396 0.79884335
 0.80032414 0.79875959 0.80001317 0.79903935 0.80035767 0.80161496
 0.79835623 0.8017062  0.80106653 0.80107182 0.80093862 0.80028798
 0.79853618 0.79869088 0.79941627]
[[1.11783256 0.81460136 1.00994133 1.00094129 1.10672736 1.04171924
  1.16900906 1.03159315 1.14672018 1.06502889 0.98391344 1.06699324
  0.98198855 1.07894744 1.0411602  1.04202094 0.85295033 0.92030859
  1.05995647 0.83602334 1.03572569 0.94181049 0.99582126 1.16310815
  0.87411457 1.17775606 1.06771578 1.06812982 1.05716336 0.98996134
  0.82181591 0.83821384 0.96242638]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, No

...

0 loss 1047982.1144559777
dev set
[0.70106992 0.69803601 0.69999009 0.69996462 0.70096308 0.70031017
 0.70179894 0.70020612 0.70135704 0.7006292  0.69975759 0.7005604
 0.69992886 0.70068074 0.70030341 0.70053023 0.69842695 0.69914295
 0.70068024 0.69847167 0.70025069 0.69930923 0.70006964 0.70153303
 0.69863144 0.70187726 0.70078803 0.70079226 0.70068175 0.70000571
 0.6983273  0.69849104 0.69967871]
[[1.11764836 0.81432052 1.00966498 1.00072374 1.10648253 1.04144346
  1.16910102 1.03131368 1.14644095 1.0648225  0.9836719  1.06671964
  0.9822736  1.07868583 1.04091554 1.04212894 0.85323448 0.92003509
  1.05972125 0.8363086  1.03548103 0.94154035 0.99610727 1.16305587
  0.87383918 1.1774532  1.06797904 1.06840093 1.05737003 0.99023637
  0.82188486 0.83827166 0.96221539]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, Non

4 loss 1048405.9627279516
dev set
[0.80072267 0.79763061 0.79958659 0.7995298  0.80056189 0.79990558
 0.80218748 0.79980076 0.80095091 0.80017286 0.79935795 0.80015534
 0.80034506 0.80029246 0.79991803 0.80093392 0.79905229 0.79871191
 0.80012238 0.79888178 0.79985303 0.79890545 0.80048112 0.8014071
 0.79822575 0.80144664 0.80119675 0.80120085 0.80108419 0.80042048
 0.79873199 0.7988764  0.79928547]
[[1.11801046 0.81472615 1.01006891 1.00111691 1.10688416 1.0418486
  1.16881549 1.0317194  1.14684743 1.0652142  0.98406264 1.06712513
  0.98186094 1.07907693 1.0413051  1.04183271 0.85282279 0.92043309
  1.06012519 0.83590073 1.03588027 0.94194444 0.9956963  1.16340405
  0.87424514 1.17796603 1.06757637 1.06799581 1.05699056 0.98982751
  0.82152967 0.83800147 0.96257756]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

3 loss 1049086.9366803495
dev set
[1.00161607 0.9977197  1.00056989 1.00027988 1.00150631 1.0008771
 1.00212647 1.00078139 1.00190008 1.00110063 0.99945386 1.00111888
 0.99957091 1.00123479 1.00087209 1.0008856  0.99826143 0.99877942
 1.00105156 0.99815461 1.00082074 0.99898199 1.00006995 1.00206512
 0.99831247 1.00220847 1.00113308 1.00113716 1.00103093 0.9996746
 0.9978066  0.99800696 0.99947857]
[[1.11791075 0.81464424 1.0095857  1.0006254  1.10672605 1.04147173
  1.16961416 1.03129988 1.14673019 1.06495487 0.98395889 1.06685329
  0.98256614 1.07886231 1.04094393 1.04237316 0.85363418 0.92036647
  1.05983611 0.83665227 1.03548439 0.9418857  0.99620554 1.16341654
  0.87416866 1.17788893 1.06833003 1.06875365 1.05767872 0.9905153
  0.82241203 0.83876047 0.96228663]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2206, 1: 2414}
acc 0.7257575757575757
(0.5492957746478874, 0.881063122923588, 0.6767032406226078, Non

2 loss 665814.7541783793
dev set
[0.70100873 0.69797491 0.69992861 0.69999614 0.70090077 0.70024605
 0.70185964 0.70014477 0.7012959  0.70057838 0.69972029 0.70049859
 0.69998879 0.70062063 0.70024146 0.70059077 0.69836659 0.69905155
 0.7007554  0.69853227 0.70018991 0.69924697 0.70013039 0.70145994
 0.69856967 0.70193967 0.70084886 0.70085305 0.70074269 0.7000696
 0.69838844 0.69855178 0.69974919]
[[1.11770987 0.81438167 1.00972655 1.00078076 1.10654392 1.04150616
  1.16904013 1.03137509 1.14650213 1.06488086 0.98372958 1.06678156
  0.98221322 1.07874588 1.04097845 1.04206884 0.85317301 0.92009741
  1.05978159 0.83624811 1.0355416  0.94160273 0.99604648 1.163172
  0.87390104 1.17746895 1.06791813 1.06834011 1.05730889 0.99017532
  0.82182285 0.83821176 0.9622744 ]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)


1 loss 666705.5291795551
dev set
[0.9010628  0.89802907 0.89998258 0.89989498 0.90095092 0.90029998
 0.90166163 0.90019894 0.90135005 0.90053559 0.89972425 0.90055279
 0.89993783 0.90067372 0.9002953  0.90039014 0.89864717 0.89908696
 0.90048416 0.89847651 0.90024177 0.89930106 0.90007556 0.90151548
 0.89862409 0.90166261 0.90078973 0.9007962  0.90067311 0.90001715
 0.89828519 0.89837467 0.89951493]
[[1.11765773 0.81432762 1.00967269 1.00073582 1.10649081 1.04145198
  1.16929958 1.03132101 1.14644803 1.06483518 0.98368237 1.06672741
  0.98226788 1.07869388 1.04092543 1.0423295  0.85322762 0.92004713
  1.05973718 0.83630568 1.03548833 0.94154867 0.99610173 1.16312624
  0.87384669 1.17759018 1.06798934 1.06839941 1.05746497 0.99023017
  0.82210109 0.83844166 0.96225852]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, Non

In [70]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=11,debug=False)

[7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2]
batch-size: 128 alpha-init: 0.6
precision and recall and sign 2 penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 15275564.851543982
dev set
[0.6004729  0.59739159 0.59936347 0.60062471 0.60116366 0.599981
 0.60246111 0.59956937 0.60072455 0.60117509 0.60037405 0.59994626
 0.59907804 0.60006834 0.59968293 0.60120344 0.59775576 0.59947162
 0.60127991 0.59914536 0.59998774 0.59870129 0.6007439  0.60095092
 0.59800611 0.60249813 0.60145662 0.60146048 0.60134149 0.59915694
 0.59898231 0.5991561  0.60017017]
[[1.11810458 0.81495594 1.01027717 1.00055709 1.10679171 1.04200729
  1.16846269 1.0319405  1.14706333 1.06467489 0.98368826 1.06731607
  0.98190972 1.07923261 1.04144353 1.04147307 0.85397106 0.92024795
  1.05943648 0.83563677 1.03594556 0.94213797 0.99543436 1.16334616
  0.87445143 1.17689676 1.06731779 1.0677388  1.05672962 0.98993095
  0.82126883 0.83764199 0.96196431]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

4 loss 15275702.309254438
dev set
[0.69876748 0.69446285 0.69651984 0.69998087 0.69875239 0.6971412
 0.70536151 0.69667701 0.6978599  0.70023846 0.69875701 0.69715124
 0.70315806 0.6977051  0.69759205 0.70417435 0.69676354 0.69724533
 0.70169494 0.70220855 0.69742059 0.6958381  0.70382367 0.7002684
 0.69510865 0.70436079 0.70450006 0.70451879 0.70429014 0.70243727
 0.70182265 0.70189938 0.70036816]
[[1.1195951  0.81788574 1.01312608 1.0025929  1.10934179 1.04483724
  1.16597837 1.03483997 1.14993898 1.06665351 0.9860397  1.07011834
  0.97853473 1.08153239 1.04335862 1.03884367 0.84942009 0.92287291
  1.06120456 0.83260286 1.03860799 0.94500547 0.99236537 1.16355902
  0.87735111 1.1775615  1.0643476  1.06471666 1.05402724 0.98677715
  0.81895054 0.83553233 0.96297296]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

[0.89980149 0.8959481  0.89794314 0.89823264 0.89908271 0.89824189
 0.90161925 0.898131   0.89928198 0.89890922 0.89791876 0.89854439
 0.90206433 0.89891647 0.89869463 0.90027115 0.90078861 0.897177
 0.89882721 0.90050445 0.89841519 0.89725859 0.90221831 0.90086592
 0.89656993 0.90004436 0.90242874 0.90271254 0.90125701 0.9019741
 0.89831285 0.89837208 0.89842093]
[[1.11908146 0.81641933 1.01172517 1.00245662 1.10839039 1.04353011
  1.16989706 1.03340145 1.14852913 1.06651726 0.98553215 1.06875
  0.98023669 1.08050189 1.04259793 1.04320056 0.85114926 0.92199773
  1.06148812 0.83449831 1.037343   0.94360251 0.99406175 1.16418069
  0.87591256 1.17936504 1.06696413 1.0667328  1.0575095  0.98845869
  0.82244722 0.83876298 0.96348454]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2975, 1: 1645}
acc 0.7489177489177489
(0.6048632218844985, 0.6611295681063123, 0.6317460317460317, None)

3 loss 15276614.230824167
dev set
[

...

[0.8008963  0.79783329 0.7997883  0.7997169  0.80075981 0.8001069
 0.80199227 0.80000339 0.80115381 0.80035804 0.79954448 0.80035778
 0.80013887 0.80048622 0.80011082 0.80073196 0.79884654 0.79890414
 0.80030956 0.79867694 0.80005083 0.79910722 0.80027562 0.80150661
 0.79842865 0.80158844 0.80099227 0.80099643 0.80088237 0.80021591
 0.79852962 0.79868249 0.7994165 ]
[[1.11783059 0.81452339 1.00986705 1.00092259 1.10668426 1.04164638
  1.16896698 1.03151663 1.14664436 1.06502156 0.98386957 1.06692253
  0.98206686 1.07888222 1.04111114 1.0420023  0.85302824 0.92023643
  1.05992702 0.83610452 1.03568105 0.94174256 0.99590158 1.16324632
  0.87404214 1.17777391 1.06777844 1.06819881 1.05718225 0.99003159
  0.82171027 0.83814446 0.96240953]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

3 loss 2281858.1060529556
dev 

1 loss 2282538.2891495028
dev set
[1.00138999 0.99793246 1.00030996 1.00005276 1.00127862 1.0006316
 1.00190565 1.00053111 1.00167729 1.00086379 0.99965376 1.0008826
 0.99969284 1.00100167 1.00062638 1.0006405  0.99841027 0.99899083
 1.00081312 0.9982611  1.00057247 0.99920191 1.00005292 1.00184379
 0.99852716 1.00198825 1.00089723 1.00090143 1.00079177 0.99978982
 0.99801169 0.998192   0.99953581]
[[1.11770474 0.81442528 1.00956161 1.00062431 1.10652564 1.04137139
  1.16939765 1.03122602 1.14651632 1.06480127 0.98374332 1.06669605
  0.98245916 1.07868684 1.04084461 1.04227046 0.85347403 0.92014419
  1.05969243 0.83653363 1.03539983 0.94165078 0.99620705 1.16320053
  0.87394522 1.17767187 1.06817048 1.06859338 1.05753952 0.99040435
  0.82220044 0.83856976 0.96222806]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2207, 1: 2413}
acc 0.7259740259740259
(0.5495234148363034, 0.881063122923588, 0.6768759571209801, No

precision and recall and sign 2 penalty
0 loss 1353298.412201915
dev set
[0.70111696 0.69808369 0.70003716 0.69998787 0.7010065  0.70035495
 0.70175062 0.70025361 0.70140476 0.70061814 0.69979213 0.70060754
 0.69988176 0.70072762 0.70034961 0.70048109 0.69847499 0.69914636
 0.70065291 0.69842281 0.7002962  0.69935582 0.70002082 0.70157147
 0.69867882 0.70183221 0.70073941 0.70074365 0.70063331 0.69996141
 0.69827864 0.69844232 0.69966847]
[[1.11760279 0.81427296 1.00961807 1.00067981 1.10643596 1.04139705
  1.16914785 1.0312663  1.14639329 1.06477939 0.9836269  1.06667263
  0.98232297 1.07863961 1.04087088 1.04217775 0.85328269 0.91999208
  1.05968032 0.83635742 1.03543419 0.9414939  0.99615603 1.16306588
  0.87379194 1.17746379 1.06802734 1.06844932 1.0574179  0.99028499
  0.82193283 0.83831959 0.96219349]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6

4 loss 1353723.6544690528
dev set
[0.80090531 0.79786734 0.79982121 0.79974274 0.80079055 0.80013763
 0.80196224 0.80003711 0.80118817 0.80038298 0.79957117 0.80039077
 0.80010113 0.80051635 0.80013585 0.80069625 0.79881037 0.79893105
 0.80033298 0.7986402  0.80008257 0.79913916 0.80023884 0.80139231
 0.79846179 0.80155219 0.80095641 0.80096061 0.80084951 0.80018041
 0.7984978  0.79865017 0.79939763]
[[1.11781651 0.81448935 1.00983409 1.00089267 1.10665199 1.04161447
  1.16896942 1.03148286 1.14660994 1.06499225 0.98383965 1.06688947
  0.98210461 1.07885157 1.04108511 1.04199579 0.85306446 0.92020593
  1.0598957  0.83614103 1.03564832 0.94171059 0.99593818 1.16328749
  0.874009   1.1777449  1.06781263 1.06823367 1.05720699 0.99006684
  0.82171332 0.83816471 0.96240521]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, No

[1.00134575 0.99797406 1.00026615 1.00006445 1.00123425 1.00058588
 1.00186182 1.00048512 1.00163331 1.00081871 0.99968676 1.00083756
 0.99971168 1.00095688 1.00058065 1.00059481 0.99839093 0.99903212
 1.00076791 0.99828016 1.00052658 0.99924463 1.00005511 1.00179992
 0.99856817 1.00194448 1.00085223 1.00085643 1.00074651 0.99981185
 0.99805314 0.99822481 0.99953081]
[[1.11768644 0.81438321 1.00956155 1.00062431 1.10651203 1.04137565
  1.16936531 1.03122848 1.14648876 1.06480477 0.98371601 1.06669904
  0.98245772 1.07868569 1.0408488  1.04227486 0.85348386 0.92010256
  1.05969691 0.83651122 1.03540308 0.94160675 0.99620134 1.16316929
  0.87390347 1.17763821 1.06817309 1.06859587 1.05754428 0.9904019
  0.82215877 0.83853684 0.96224041]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2207, 1: 2413}
acc 0.7259740259740259
(0.5495234148363034, 0.881063122923588, 0.6768759571209801, None)

3 loss 1354402.9837810246
de

In [68]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=7,debug=False)

[7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2]
batch-size: 128 alpha-init: 0.6
precision and recall penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 15275564.851543982
dev set
[0.6004729  0.59739159 0.59936347 0.60062471 0.60116366 0.599981
 0.60246111 0.59956937 0.60072455 0.60117509 0.60037405 0.59994626
 0.59907804 0.60006834 0.59968293 0.60120344 0.59775576 0.59947162
 0.60127991 0.59914536 0.59998774 0.59870129 0.6007439  0.60095092
 0.59800611 0.60249813 0.60145662 0.60146048 0.60134149 0.59915694
 0.59898231 0.5991561  0.60017017]
[[1.11810458 0.81495594 1.01027717 1.00055709 1.10679171 1.04200729
  1.16846269 1.0319405  1.14706333 1.06467489 0.98368826 1.06731607
  0.98190972 1.07923261 1.04144353 1.04147307 0.85397106 0.92024795
  1.05943648 0.83563677 1.03594556 0.94213797 0.99543436 1.16334616
  0.87445143 1.17689676 1.06731779 1.0677388  1.05672962 0.98993095
  0.82126883 0.83764199 0.96196431]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

4 loss 15275702.309254438
dev set
[0.69876748 0.69446285 0.69651984 0.69998087 0.69875239 0.6971412
 0.70536151 0.69667701 0.6978599  0.70023846 0.69875701 0.69715124
 0.70315806 0.6977051  0.69759205 0.70417435 0.69676354 0.69724533
 0.70169494 0.70220855 0.69742059 0.6958381  0.70382367 0.7002684
 0.69510865 0.70436079 0.70450006 0.70451879 0.70429014 0.70243727
 0.70182265 0.70189938 0.70036816]
[[1.1195951  0.81788574 1.01312608 1.0025929  1.10934179 1.04483724
  1.16597837 1.03483997 1.14993898 1.06665351 0.9860397  1.07011834
  0.97853473 1.08153239 1.04335862 1.03884367 0.84942009 0.92287291
  1.06120456 0.83260286 1.03860799 0.94500547 0.99236537 1.16355902
  0.87735111 1.1775615  1.0643476  1.06471666 1.05402724 0.98677715
  0.81895054 0.83553233 0.96297296]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test s

2 loss 15276617.409518797
dev set
[0.89980149 0.8959481  0.89794314 0.89823264 0.89908271 0.89824189
 0.90161925 0.898131   0.89928198 0.89890922 0.89791876 0.89854439
 0.90206433 0.89891647 0.89869463 0.90027115 0.90078861 0.897177
 0.89882721 0.90050445 0.89841519 0.89725859 0.90221831 0.90086592
 0.89656993 0.90004436 0.90242874 0.90271254 0.90125701 0.9019741
 0.89831285 0.89837208 0.89842093]
[[1.11908146 0.81641933 1.01172517 1.00245662 1.10839039 1.04353011
  1.16989706 1.03340145 1.14852913 1.06651726 0.98553215 1.06875
  0.98023669 1.08050189 1.04259793 1.04320056 0.85114926 0.92199773
  1.06148812 0.83449831 1.037343   0.94360251 0.99406175 1.16418069
  0.87591256 1.17936504 1.06696413 1.0667328  1.0575095  0.98845869
  0.82244722 0.83876298 0.96348454]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2975, 1: 1645}
acc 0.7489177489177489
(0.6048632218844985, 0.6611295681063123, 0.6317460317460317, None)

3

...

2 loss 2281858.8991301134
dev set
[0.8008963  0.79783329 0.7997883  0.7997169  0.80075981 0.8001069
 0.80199227 0.80000339 0.80115381 0.80035804 0.79954448 0.80035778
 0.80013887 0.80048622 0.80011082 0.80073196 0.79884654 0.79890414
 0.80030956 0.79867694 0.80005083 0.79910722 0.80027562 0.80150661
 0.79842865 0.80158844 0.80099227 0.80099643 0.80088237 0.80021591
 0.79852962 0.79868249 0.7994165 ]
[[1.11783059 0.81452339 1.00986705 1.00092259 1.10668426 1.04164638
  1.16896698 1.03151663 1.14664436 1.06502156 0.98386957 1.06692253
  0.98206686 1.07888222 1.04111114 1.0420023  0.85302824 0.92023643
  1.05992702 0.83610452 1.03568105 0.94174256 0.99590158 1.16324632
  0.87404214 1.17777391 1.06777844 1.06819881 1.05718225 0.99003159
  0.82171027 0.83814446 0.96240953]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, Non

1 loss 2282538.2891495028
dev set
[1.00138999 0.99793246 1.00030996 1.00005276 1.00127862 1.0006316
 1.00190565 1.00053111 1.00167729 1.00086379 0.99965376 1.0008826
 0.99969284 1.00100167 1.00062638 1.0006405  0.99841027 0.99899083
 1.00081312 0.9982611  1.00057247 0.99920191 1.00005292 1.00184379
 0.99852716 1.00198825 1.00089723 1.00090143 1.00079177 0.99978982
 0.99801169 0.998192   0.99953581]
[[1.11770474 0.81442528 1.00956161 1.00062431 1.10652564 1.04137139
  1.16939765 1.03122602 1.14651632 1.06480127 0.98374332 1.06669605
  0.98245916 1.07868684 1.04084461 1.04227046 0.85347403 0.92014419
  1.05969243 0.83653363 1.03539983 0.94165078 0.99620705 1.16320053
  0.87394522 1.17767187 1.06817048 1.06859338 1.05753952 0.99040435
  0.82220044 0.83856976 0.96222806]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2207, 1: 2413}
acc 0.7259740259740259
(0.5495234148363034, 0.881063122923588, 0.6768759571209801, No

precision and recall penalty
0 loss 1353298.412201915
dev set
[0.70111696 0.69808369 0.70003716 0.69998787 0.7010065  0.70035495
 0.70175062 0.70025361 0.70140476 0.70061814 0.69979213 0.70060754
 0.69988176 0.70072762 0.70034961 0.70048109 0.69847499 0.69914636
 0.70065291 0.69842281 0.7002962  0.69935582 0.70002082 0.70157147
 0.69867882 0.70183221 0.70073941 0.70074365 0.70063331 0.69996141
 0.69827864 0.69844232 0.69966847]
[[1.11760279 0.81427296 1.00961807 1.00067981 1.10643596 1.04139705
  1.16914785 1.0312663  1.14639329 1.06477939 0.9836269  1.06667263
  0.98232297 1.07863961 1.04087088 1.04217775 0.85328269 0.91999208
  1.05968032 0.83635742 1.03543419 0.9414939  0.99615603 1.16306588
  0.87379194 1.17746379 1.06802734 1.06844932 1.0574179  0.99028499
  0.82193283 0.83831959 0.96219349]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.661129568106

4 loss 1353723.6544690528
dev set
[0.80090531 0.79786734 0.79982121 0.79974274 0.80079055 0.80013763
 0.80196224 0.80003711 0.80118817 0.80038298 0.79957117 0.80039077
 0.80010113 0.80051635 0.80013585 0.80069625 0.79881037 0.79893105
 0.80033298 0.7986402  0.80008257 0.79913916 0.80023884 0.80139231
 0.79846179 0.80155219 0.80095641 0.80096061 0.80084951 0.80018041
 0.7984978  0.79865017 0.79939763]
[[1.11781651 0.81448935 1.00983409 1.00089267 1.10665199 1.04161447
  1.16896942 1.03148286 1.14660994 1.06499225 0.98383965 1.06688947
  0.98210461 1.07885157 1.04108511 1.04199579 0.85306446 0.92020593
  1.0598957  0.83614103 1.03564832 0.94171059 0.99593818 1.16328749
  0.874009   1.1777449  1.06781263 1.06823367 1.05720699 0.99006684
  0.82171332 0.83816471 0.96240521]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, No

[1.00134575 0.99797406 1.00026615 1.00006445 1.00123425 1.00058588
 1.00186182 1.00048512 1.00163331 1.00081871 0.99968676 1.00083756
 0.99971168 1.00095688 1.00058065 1.00059481 0.99839093 0.99903212
 1.00076791 0.99828016 1.00052658 0.99924463 1.00005511 1.00179992
 0.99856817 1.00194448 1.00085223 1.00085643 1.00074651 0.99981185
 0.99805314 0.99822481 0.99953081]
[[1.11768644 0.81438321 1.00956155 1.00062431 1.10651203 1.04137565
  1.16936531 1.03122848 1.14648876 1.06480477 0.98371601 1.06669904
  0.98245772 1.07868569 1.0408488  1.04227486 0.85348386 0.92010256
  1.05969691 0.83651122 1.03540308 0.94160675 0.99620134 1.16316929
  0.87390347 1.17763821 1.06817309 1.06859587 1.05754428 0.9904019
  0.82215877 0.83853684 0.96224041]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2207, 1: 2413}
acc 0.7259740259740259
(0.5495234148363034, 0.881063122923588, 0.6768759571209801, None)

3 loss 1354402.9837810246
de

In [71]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=12,debug=False)

[7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2]
batch-size: 128 alpha-init: 0.6
recall and sign 2 penalty
0 loss 15269861.522691036
dev set
[0.6004729  0.59739159 0.59936347 0.60062471 0.60116366 0.599981
 0.60246111 0.59956937 0.60072455 0.60117509 0.60037405 0.59994626
 0.59907804 0.60006834 0.59968293 0.60120344 0.59775576 0.59947162
 0.60127991 0.59914536 0.59998774 0.59870129 0.6007439  0.60095092
 0.59800611 0.60249813 0.60145662 0.60146048 0.60134149 0.59915694
 0.59898231 0.5991561  0.60017017]
[[1.11810458 0.81495594 1.01027717 1.00055709 1.10679171 1.04200729
  1.16846269 1.0319405  1.14706333 1.06467489 0.98368826 1.06731607
  0.98190972 1.07923261 1.04144353 1.04147307 0.85397106 0.92024795
  1.05943648 0.83563677 1.03594556 0.94213797 0.99543436 1.16334616
  0.87445143 1.17689676 1.067317

3 loss 15270006.30169283
dev set
[0.69921198 0.69519721 0.69723345 0.69999699 0.69922476 0.69779739
 0.70463791 0.69740182 0.69857676 0.70033934 0.69898583 0.69785257
 0.7023739  0.69829846 0.69811764 0.70343688 0.69681885 0.69766084
 0.7014999  0.70144886 0.69801982 0.69655716 0.70305983 0.7004905
 0.69583635 0.70386856 0.70374368 0.70375765 0.70355299 0.70184583
 0.70111161 0.7012114  0.70023855]
[[1.11921026 0.81715068 1.01241152 1.00218696 1.10874696 1.04413663
  1.16661044 1.03411314 1.14921933 1.0662529  0.98554584 1.06941617
  0.97931776 1.08095913 1.04288074 1.03950785 0.85024054 0.92228974
  1.06089344 0.8333554  1.03795205 0.94428541 0.99312642 1.16346474
  0.87662291 1.17750559 1.06508742 1.06547048 1.05471005 0.98750022
  0.8195482  0.8360818  0.96280235]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

2 loss 15270914.08066522
dev set
[0.89980149 0.8959481  0.89794314 0.89823264 0.89908271 0.89824189
 0.90161925 0.898131   0.89928198 0.89890922 0.89791876 0.89854439
 0.90206433 0.89891647 0.89869463 0.90027115 0.90078861 0.897177
 0.89882721 0.90050445 0.89841519 0.89725859 0.90221831 0.90086592
 0.89656993 0.90004436 0.90242874 0.90271254 0.90125701 0.9019741
 0.89831285 0.89837208 0.89842093]
[[1.11908146 0.81641933 1.01172517 1.00245662 1.10839039 1.04353011
  1.16989706 1.03340145 1.14852913 1.06651726 0.98553215 1.06875
  0.98023669 1.08050189 1.04259793 1.04320056 0.85114926 0.92199773
  1.06148812 0.83449831 1.037343   0.94360251 0.99406175 1.16418069
  0.87591256 1.17936504 1.06696413 1.0667328  1.0575095  0.98845869
  0.82244722 0.83876298 0.96348454]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2975, 1: 1645}
acc 0.7489177489177489
(0.6048632218844985, 0.6611295681063123, 0.6317460317460317, None)

3 

{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

1 loss 7844910.824511355
dev set
[0.60044061 0.59736602 0.59933021 0.60069492 0.60118411 0.59990048
 0.60247536 0.59953988 0.60069447 0.60125008 0.60042628 0.59990467
 0.5990557  0.60002746 0.59965468 0.60121361 0.59774888 0.5995235
 0.60132656 0.59915348 0.59989926 0.59866217 0.60075225 0.60089764
 0.59797018 0.6025313  0.60146786 0.60147205 0.6013575  0.5991374
 0.59899975 0.5991676  0.60027841]
[[1.11817211 0.81498659 1.01031773 1.00051541 1.10689426 1.04205182
  1.1684377  1.03197547 1.14709871 1.06467358 0.98372709 1.06736685
  0.98178406 1.07930423 1.04150313 1.04145167 0.85402554 0.92039422
  1.05936297 0.83562805 1.0360133  0.94218135 0.9954252  1.16347486
  0.87449437 1.1768267  1.06730295 1.06772397 1.05670367 0.98986877
  0.82123577 0.83761575 0.96189112]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test se

4 loss 7845067.29196695
dev set
[0.69971963 0.69622267 0.69820893 0.70001282 0.69970014 0.69864633
 0.70359989 0.69840289 0.69956291 0.70040827 0.69917934 0.69879186
 0.70157309 0.69902588 0.69874357 0.70236582 0.69705463 0.69795068
 0.70140189 0.70032269 0.69875078 0.6975352  0.70192574 0.7006097
 0.6968355  0.7033247  0.70263217 0.7026388  0.70249917 0.70125301
 0.70010758 0.70024532 0.70022724]
[[1.11881952 0.81613188 1.0114432  1.00183202 1.10801922 1.04318367
  1.16743979 1.03311618 1.14823521 1.06587494 0.9850061  1.06848493
  0.98041379 1.08029724 1.04237055 1.04038572 0.85135656 0.92164081
  1.0606671  0.83446598 1.03711977 0.94331209 0.9942542  1.16346152
  0.87563191 1.17749208 1.06615679 1.06656677 1.05562359 0.98847928
  0.82029654 0.83675982 0.96281777]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

...

recall and sign 2 penalty
0 loss 2276374.114455978
dev set
[0.70106992 0.69803601 0.69999009 0.69996462 0.70096308 0.70031017
 0.70179894 0.70020612 0.70135704 0.7006292  0.69975759 0.7005604
 0.69992886 0.70068074 0.70030341 0.70053023 0.69842695 0.69914295
 0.70068024 0.69847167 0.70025069 0.69930923 0.70006964 0.70153303
 0.69863144 0.70187726 0.70078803 0.70079226 0.70068175 0.70000571
 0.6983273  0.69849104 0.69967871]
[[1.11764836 0.81432052 1.00966498 1.00072374 1.10648253 1.04144346
  1.16910102 1.03131368 1.14644095 1.0648225  0.9836719  1.06671964
  0.9822736  1.07868583 1.04091554 1.04212894 0.85323448 0.92003509
  1.05972125 0.8363086  1.03548103 0.94154035 0.99610727 1.16305587
  0.87383918 1.1774532  1.06797904 1.06840093 1.05737003 0.99023637
  0.82188486 0.83827166 0.96221539]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123

recall and sign 2 penalty
0 loss 2277264.0793753252
dev set
[0.90107115 0.89803629 0.89999015 0.89990165 0.90095766 0.90030752
 0.90165651 0.90020627 0.9013571  0.90054206 0.89973053 0.90056042
 0.89993204 0.90068077 0.90030379 0.90038569 0.89864062 0.89909432
 0.90049106 0.89847083 0.90024881 0.89930898 0.90006905 0.90153926
 0.8986317  0.90166904 0.90078178 0.90078905 0.90066084 0.90001062
 0.89821737 0.89837279 0.89952704]
[[1.11764999 0.81432041 1.00966515 1.0007292  1.10648411 1.04144453
  1.16928794 1.0313137  1.14644099 1.06482876 0.98367609 1.06671982
  0.98227358 1.07868695 1.04091731 1.04232212 0.85323438 0.92003997
  1.05973043 0.83631055 1.03548136 0.94154078 0.9961084  1.16311123
  0.8738391  1.17758402 1.06799891 1.06840687 1.05744137 0.99023703
  0.82207768 0.83844017 0.96224732]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.66112956810631

recall and sign 2 penalty
0 loss 1348091.25645172
dev set
[0.60111672 0.59808363 0.60003709 0.60006515 0.60110657 0.60035537
 0.60175071 0.60025355 0.60140473 0.60070519 0.59989252 0.60060752
 0.59976531 0.60072735 0.60034949 0.60048112 0.59847472 0.59924512
 0.60065511 0.59842283 0.60029823 0.59935581 0.60002083 0.60157122
 0.59867877 0.60183321 0.60073948 0.60074371 0.60063339 0.59984483
 0.59827862 0.59844248 0.59967935]
[[1.11760309 0.81427298 1.00961809 1.00062108 1.10643492 1.04139699
  1.16914745 1.03126632 1.1463933  1.06471552 0.98360508 1.06667264
  0.98232447 1.07863974 1.04087097 1.04217739 0.85339976 0.91998785
  1.05956962 0.83635737 1.03543398 0.94149389 0.99615601 1.16306979
  0.87379196 1.17741996 1.06802717 1.0684492  1.05741772 0.99028605
  0.82193285 0.83831887 0.96210376]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123

recall and sign 2 penalty
0 loss 1348682.9422051057
dev set
[0.80111742 0.79808372 0.80003717 0.79994968 0.80100562 0.80035487
 0.8017499  0.80025362 0.80140477 0.8005902  0.79977876 0.80060755
 0.79988268 0.80072774 0.8003497  0.80048049 0.79859209 0.79914161
 0.80053974 0.79842274 0.80029589 0.79935581 0.80002079 0.80158381
 0.79867885 0.80172471 0.80073916 0.80074346 0.800633   0.79996215
 0.79827868 0.79844073 0.79957438]
[[1.11760286 0.81427296 1.00961808 1.00068162 1.10643616 1.04139708
  1.1691534  1.03126631 1.14639329 1.06478097 0.98362822 1.06667264
  0.98232298 1.07863971 1.04087092 1.04218439 0.85328266 0.9199927
  1.05968252 0.83635757 1.03543424 0.94149392 0.99615609 1.16306819
  0.87379192 1.17753528 1.06802802 1.0684497  1.0574189  0.99028501
  0.82193287 0.83832808 0.96220453]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.661129568106312

recall and sign 2 penalty
0 loss 1349360.5401437287
dev set
[1.00123421 0.9980837  1.00015233 1.0000112  1.00112255 1.00047215
 1.00175081 1.00037078 1.00152211 1.00070602 0.99978204 1.00072493
 0.99978098 1.0008446  1.00046688 1.00048112 0.99848238 0.99914106
 1.00065503 0.99833643 1.00041249 0.99935567 1.00000826 1.00168887
 0.99867869 1.00183353 1.00073964 1.00074386 1.00063355 0.99986447
 0.99816162 0.99832781 0.999591  ]
[[1.1175891  0.81427299 1.00955979 1.00062431 1.1064188  1.04134575
  1.16925916 1.03121129 1.14638517 1.06474345 0.98362079 1.0666354
  0.98240598 1.07860991 1.04081949 1.04224373 0.8533904  0.91999268
  1.0596419  0.8364496  1.03538103 0.94149412 0.99620871 1.16306369
  0.8737921  1.17753143 1.0681079  1.06853021 1.05749207 0.99036208
  0.82204976 0.83843333 0.96218019]]
{0: 438, 1: 450}
acc 0.7387387387387387
(0.5711111111111111, 0.8682432432432432, 0.6890080428954424, None)

test set
{0: 2207, 1: 2413}
acc 0.7259740259740259
(0.5495234148363034, 0.881063122923

In [58]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=9,debug=False)

[7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2]
batch-size: 128 alpha-init: 0.6
precision and sign 2 penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 193640.85154398024
dev set
[0.6004729  0.59739159 0.59936347 0.60062471 0.60116366 0.599981
 0.60246111 0.59956937 0.60072455 0.60117509 0.60037405 0.59994626
 0.59907804 0.60006834 0.59968293 0.60120344 0.59775576 0.59947162
 0.60127991 0.59914536 0.59998774 0.59870129 0.6007439  0.60095092
 0.59800611 0.60249813 0.60145662 0.60146048 0.60134149 0.59915694
 0.59898231 0.5991561  0.60017017]
[[1.11810458 0.81495594 1.01027717 1.00055709 1.10679171 1.04200729
  1.16846269 1.0319405  1.14706333 1.06467489 0.98368826 1.06731607
  0.98190972 1.07923261 1.04144353 1.04147307 0.85397106 0.92024795
  1.05943648 0.83563677 1.03594556 0.94213797 0.99543436 1.16334616
  0.87445143 1.17689676 1.06731779 1.0677388  1.05672962 0.98993095
  0.82126883 0.83764199 0.96196431]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

4 loss 193778.30925443638
dev set
[0.69876748 0.69446285 0.69651984 0.69998087 0.69875239 0.6971412
 0.70536151 0.69667701 0.6978599  0.70023846 0.69875701 0.69715124
 0.70315806 0.6977051  0.69759205 0.70417435 0.69676354 0.69724533
 0.70169494 0.70220855 0.69742059 0.6958381  0.70382367 0.7002684
 0.69510865 0.70436079 0.70450006 0.70451879 0.70429014 0.70243727
 0.70182265 0.70189938 0.70036816]
[[1.1195951  0.81788574 1.01312608 1.0025929  1.10934179 1.04483724
  1.16597837 1.03483997 1.14993898 1.06665351 0.9860397  1.07011834
  0.97853473 1.08153239 1.04335862 1.03884367 0.84942009 0.92287291
  1.06120456 0.83260286 1.03860799 0.94500547 0.99236537 1.16355902
  0.87735111 1.1775615  1.0643476  1.06471666 1.05402724 0.98677715
  0.81895054 0.83553233 0.96297296]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

3 loss 194690.2308241665
dev set
[0.89936063 0.89521145 0.89721957 0.89762992 0.89841976 0.89751748
 0.90158382 0.89739901 0.89855208 0.89832433 0.89728891 0.89783066
 0.9027992  0.89830997 0.89815292 0.90022872 0.90152814 0.89651993
 0.89824895 0.90119112 0.89775451 0.89653131 0.90296027 0.9005942
 0.89583766 0.89948454 0.90299375 0.90339863 0.90145964 0.90264181
 0.89833569 0.89835214 0.89802644]
[[1.11957966 0.81716396 1.01245759 1.00308271 1.10906779 1.04426588
  1.17013938 1.03414223 1.1492681  1.06712364 0.9861809  1.06947288
  0.97953406 1.08112993 1.04316836 1.04352599 0.8504355  0.92267201
  1.0621037  0.83391143 1.03801659 0.94433795 0.99336107 1.16457139
  0.87665314 1.17997105 1.06662588 1.06613241 1.05754177 0.98786226
  0.82261219 0.83890021 0.96391732]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2975, 1: 1645}
acc 0.7489177489177489
(0.6048632218844985, 0.6611295681063123, 0.6317460317460317, None

...

3 loss 193591.70605295416
dev set
[0.80080932 0.79773199 0.79968749 0.79962344 0.80066085 0.80000631
 0.80208994 0.79990211 0.80105239 0.8002654  0.79945116 0.8002566
 0.80024197 0.80038927 0.80001436 0.800833   0.79894937 0.79880797
 0.80021609 0.79877935 0.79995195 0.79900638 0.80037838 0.80145665
 0.79832725 0.80151756 0.8010945  0.80109862 0.80098324 0.80031817
 0.79863086 0.79877945 0.79935103]
[[1.11792064 0.81462472 1.00996792 1.00101965 1.1067842  1.04174742
  1.16889109 1.03161797 1.14674586 1.0651179  0.98396613 1.06702378
  0.98196388 1.07897962 1.04120815 1.04191741 0.85292553 0.92033481
  1.06002601 0.83600263 1.03578063 0.94184344 0.99579893 1.16332523
  0.87414358 1.17787002 1.06767742 1.06809734 1.05708646 0.98992955
  0.82161982 0.83807283 0.9624935 ]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, Non

2 loss 194271.88807571356
dev set
[1.00150036 0.99782739 1.00043336 1.00015048 1.0013896  1.00074951
 1.00201406 1.00065096 1.00178644 1.00097827 0.99955731 1.00099686
 0.99963297 1.00111471 1.00074437 1.00075825 0.99833676 0.99888658
 1.00092823 0.99820739 1.0006915  0.99909384 1.00006468 1.00195239
 0.99842123 1.00209645 1.00101133 1.00101548 1.00090716 0.99973821
 0.99790955 0.99809987 0.99950363]
[[1.11780255 0.81453262 1.00956801 1.00062438 1.10662001 1.04141099
  1.16950249 1.03125295 1.14661923 1.06486873 0.98384516 1.0667655
  0.98250872 1.07876654 1.04088369 1.04231123 0.85355236 0.92025239
  1.05975449 0.83659292 1.03543178 0.94176453 0.99620597 1.16330497
  0.87405439 1.17777717 1.06824123 1.06866453 1.05759916 0.99045122
  0.82230492 0.83866394 0.96226082]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2206, 1: 2414}
acc 0.7257575757575757
(0.5492957746478874, 0.881063122923588, 0.6767032406226078, N

{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

1 loss 193149.79321039055
dev set
[0.70106254 0.69802902 0.69998259 0.69999057 0.70095332 0.70030023
 0.70180542 0.7001989  0.70135003 0.70059818 0.6997559  0.70055277
 0.69993543 0.70067377 0.7002952  0.70053622 0.69842041 0.69909877
 0.70070437 0.69847783 0.70024274 0.6993011  0.70007589 0.70151547
 0.69862397 0.70188618 0.70079442 0.70079863 0.70068828 0.70001569
 0.69833382 0.69849732 0.69970899]
[[1.11765664 0.8143276  1.0096726  1.0007306  1.10649024 1.04145189
  1.16909367 1.03132098 1.14644801 1.0648305  0.98367858 1.06672739
  0.98226781 1.07869311 1.04092501 1.04212302 0.8532276  0.92004505
  1.05973126 0.83630248 1.03548821 0.94154861 0.99610097 1.16311906
  0.87384677 1.17746527 1.06797247 1.06839444 1.05736312 0.99022988
  0.82187756 0.83826542 0.96223406]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test

4 loss 193575.65446905137
dev set
[0.80090531 0.79786734 0.79982121 0.79974274 0.80079055 0.80013763
 0.80196224 0.80003711 0.80118817 0.80038298 0.79957117 0.80039077
 0.80010113 0.80051635 0.80013585 0.80069625 0.79881037 0.79893105
 0.80033298 0.7986402  0.80008257 0.79913916 0.80023884 0.80139231
 0.79846179 0.80155219 0.80095641 0.80096061 0.80084951 0.80018041
 0.7984978  0.79865017 0.79939763]
[[1.11781651 0.81448935 1.00983409 1.00089267 1.10665199 1.04161447
  1.16896942 1.03148286 1.14660994 1.06499225 0.98383965 1.06688947
  0.98210461 1.07885157 1.04108511 1.04199579 0.85306446 0.92020593
  1.0598957  0.83614103 1.03564832 0.94171059 0.99593818 1.16328749
  0.874009   1.1777449  1.06781263 1.06823367 1.05720699 0.99006684
  0.82171332 0.83816471 0.96240521]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, No

3 loss 194254.98378102345
dev set
[1.00140252 0.99791899 1.00032756 1.00011493 1.00129123 1.00064534
 1.00191793 1.00054532 1.00168966 1.00087691 0.99963693 1.00089568
 0.99967717 1.00101457 1.00064013 1.00065421 0.99834496 0.99897722
 1.00082634 0.99825184 1.00058647 0.99918848 1.00006552 1.0018561
 0.99851258 1.00200052 1.00091029 1.00091448 1.00080504 0.99978111
 0.99799913 0.99817356 0.99950019]
[[1.1177388  0.81443903 1.00956438 1.00062435 1.10656306 1.04139909
  1.16942024 1.03124402 1.14654301 1.0648437  0.9837678  1.06673896
  0.98248741 1.07873048 1.04087191 1.042299   0.85353161 0.92015884
  1.05973307 0.83654238 1.03542179 0.94166485 0.99619973 1.16322408
  0.87396014 1.17769329 1.06821366 1.06863663 1.05757916 0.99042951
  0.82221356 0.83858885 0.96227176]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2207, 1: 2413}
acc 0.7259740259740259
(0.5495234148363034, 0.881063122923588, 0.6768759571209801, N

In [34]:
train(0.001,5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=False,smooth=False)

k Tensor("Const:0", shape=(33,), dtype=float64)
s Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -31043.570042320927
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-0

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [37]:
for i in np.linspace(0,1,11):
    print("alpha-init:",i)
    train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=True,penalty=1)

alpha-init: 0.0
k Tensor("Const:0", shape=(33,), dtype=float64)
s Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty1
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 159150.60326214734
dev set
[-0.00447464 -0

2 loss 146101.8036713557
dev set
[ 0.31783772  0.12210608  0.16096265  0.07221402  0.12177342  0.22570068
  0.22900776  0.14746126  0.15552667  0.09578382  0.06730813  0.16588696
  0.31022997 -0.1219712   0.31358334  0.0862623   0.2879223   0.21471383
  0.21788145  0.08852977  0.24331594 -0.08381277  0.15871157  0.14717002
  0.11021917  0.22960748  0.20777265  0.20645925  0.21663016 -0.12484925
  0.33888476  0.33928117  0.21505989]
[[0.87344591 0.82119844 0.97998965 0.73836648 0.84082826 0.91751781
  1.01236926 1.01551045 1.11005237 0.7998638  0.72228021 1.01986777
  0.74581637 1.29409924 0.80093347 0.77811016 0.65723993 0.76198245
  0.8820485  0.66482297 0.89648113 1.13800492 0.79424637 1.22812881
  0.89258996 0.99032052 0.89301669 0.88272452 0.90918854 1.20914809
  0.58456176 0.59979711 0.84069332]]
{0: 5, 1: 883}
acc 0.33896396396396394
(0.3352208380520951, 1.0, 0.5021204410517388, None)

test set
{0: 21, 1: 4599}
acc 0.3303030303030303
(0.3272450532724505, 1.0, 0.49311926605504586,

zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty1
loss Tensor("add:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 152929.69648361602
dev set
[0.34575332 0.26982512 0.28076967 0.38056177 0.38271039 0.34848991
 0.37523786 0.27811541 0.28173941 0.38188432 0.38019838 0.2829926
 0.33669091 0.24111681 0.34068412 0.38142778 0.31237052 0.35516085
 0.36267559 0.37531022 0.34613929 0.23299208 0.37932756 0.27785258
 0.26983576 0.36560141 0.37787778 0.37927258 0.37398996 0.23953309
 0.36633358 0.36658077 0.35389949]
[[1.04167205 0.8386411  1.0253237  0.91703365 1.02198585 1.00453611
  1.10173465 1.05028927 1.16024451 0.9806308  0.9001227  1.07812181
  0.9158368  1.13302546 0.97017352 0.95831374 0.82562812 0.84551592
  0.97936118 0.75761676 1.00076672 1.00749083 0.91540728 1.18613792
  0.899365

2 loss 144810.15056910855
dev set
[0.6594196  0.38817586 0.42106958 0.34875003 0.37465603 0.5117076
 0.50392271 0.4079354  0.41462119 0.36346557 0.34521724 0.42617073
 0.64989027 0.16703073 0.65409465 0.35800777 0.59523577 0.50208663
 0.5027488  0.39035249 0.5276903  0.20302956 0.44601799 0.4041976
 0.37683686 0.50133293 0.48706558 0.48233418 0.49695434 0.16480011
 0.63406435 0.63406933 0.49874955]
[[0.83212375 0.85204274 1.02100308 0.74743325 0.84404119 0.93484888
  1.05807468 1.05469323 1.15024911 0.80487415 0.7331902  1.06045141
  0.70667524 1.29842203 0.76085805 0.78422294 0.67255557 0.77106018
  0.85242716 0.71892552 0.91405828 1.14895949 0.84923287 1.22881745
  0.92278817 0.94255128 0.94111128 0.937555   0.94770344 1.21778531
  0.60455481 0.6157577  0.86560158]]
{0: 2, 1: 886}
acc 0.3355855855855856
(0.3340857787810384, 1.0, 0.5008460236886634, None)

test set
{0: 8, 1: 4612}
acc 0.3274891774891775
(0.32632263660017347, 1.0, 0.49207127676965834, None)

3 loss 140477.074046309
dev

0 loss 151070.04968550202
dev set
[0.7100312  0.57418233 0.5824313  0.68044697 0.6827378  0.62771296
 0.6535066  0.57969128 0.58607797 0.68186428 0.68005313 0.58783509
 0.69632263 0.55669071 0.70261765 0.68137777 0.63293874 0.67851152
 0.68175493 0.66942482 0.62303135 0.53824994 0.67353768 0.57802203
 0.57378252 0.68415518 0.66441633 0.66645546 0.65417283 0.54929844
 0.65369588 0.65333552 0.62843143]
[[1.01091522 0.83921447 1.02895548 0.91831226 1.02325235 1.01939494
  1.12932677 1.05355753 1.16394247 0.98189872 0.90140628 1.08156152
  0.87044758 1.11958062 0.93159832 0.95958411 0.80732299 0.83836977
  0.97683873 0.76822063 1.01802121 1.00383745 0.92632396 1.18525391
  0.89995565 1.0938882  1.0131493  1.01244895 1.01540976 1.03276681
  0.77442174 0.78369828 0.94072303]]
{0: 179, 1: 709}
acc 0.49436936936936937
(0.3921015514809591, 0.9391891891891891, 0.5532338308457712, None)

test set
{0: 948, 1: 3672}
acc 0.5175324675324675
(0.4014161220043573, 0.9794019933554817, 0.5694417616380142,

3 loss 146163.9236450166
dev set
[1.         0.62506072 0.64987757 0.53331341 0.55911585 0.81375832
 0.77469006 0.6376841  0.65779355 0.54812453 0.52973111 0.66884367
 1.         0.44141386 1.         0.54266264 0.87460684 0.51741003
 0.54685873 0.7054032  0.82623251 0.43711751 0.7479184  0.62878057
 0.61495215 0.58075862 0.76803526 0.76414151 0.76979525 0.44152815
 0.8111072  0.79971356 0.78905393]
[[0.87141684 0.9056346  1.08081944 0.62907489 0.729309   0.93024903
  1.09991771 1.11425164 1.21238929 0.68905895 0.61374113 1.12157744
  0.73765931 1.29806423 0.79509189 0.66768882 0.73357813 0.56203542
  0.68418881 0.75447004 0.91274218 1.21506976 0.89953036 1.24981215
  0.97586762 0.7995112  0.98838775 0.99436398 0.98935591 1.17801881
  0.74452309 0.75363397 0.87432198]]
{1: 888}
acc 0.3333333333333333
(0.3333333333333333, 1.0, 0.5, None)

test set
{0: 2, 1: 4618}
acc 0.3261904761904762
(0.32589865742745777, 1.0, 0.4915890903152049, None)

4 loss 143204.1860546232
dev set
[1.         0.6

0 loss 151878.4451930209
dev set
[0.98143189 0.87048724 0.87547943 0.95736725 0.96006688 0.92049363
 0.91134971 0.87361129 0.87748489 0.95906963 0.9568743  0.87930833
 0.98119868 0.87922882 0.98128172 0.95849746 0.92699809 0.95485433
 0.9589421  0.92865549 0.91622594 0.85183945 0.93582332 0.87149888
 0.87121691 0.96160889 0.9230289  0.91356621 0.90769222 0.9095905
 0.90729866 0.91719953 0.90982374]
[[1.0555966  0.84298243 1.03531526 0.94460283 1.05035843 1.02189767
  1.16147835 1.05912165 1.17150831 1.00869034 0.92755956 1.08909424
  0.91945621 1.10108257 0.97831781 0.98620248 0.832654   0.86399335
  1.00359129 0.81151755 1.02005317 0.99084238 0.96435688 1.18522444
  0.90237032 1.12151398 1.04836467 1.05879104 1.05211369 0.98686294
  0.81485175 0.82418017 0.95251222]]
{0: 342, 1: 546}
acc 0.6554054054054054
(0.4908424908424908, 0.9054054054054054, 0.6365795724465558, None)

test set
{0: 1778, 1: 2842}
acc 0.6811688311688312
(0.5056298381421535, 0.9548172757475083, 0.6611456176673568, N

3 loss 147140.4527114103
dev set
[1.48857286 0.85332978 1.48898104 1.         1.47849378 1.47838197
 1.47858942 1.4889911  1.4884422  1.4784269  0.99999997 1.48882138
 0.55264483 1.48876152 1.48894861 1.47838395 0.87366752 0.9999999
 1.47841793 0.99999999 1.47836785 0.82818621 1.         1.48328827
 0.90465087 1.47860218 1.47843266 1.47843338 1.47841405 0.91392069
 1.         1.00007363 1.        ]
[[1.62204947 0.96097956 1.51351832 1.00062428 1.6028828  1.53738981
  1.66610418 1.53548192 1.65090317 1.56095194 0.9835706  1.57100626
  0.76442603 1.58300145 1.54512995 1.53829496 0.92925157 0.9199338
  1.55581732 0.83640691 1.53137417 1.12596628 0.98985765 1.25032081
  0.97151689 1.67442583 1.56433775 1.56476272 1.55365323 1.06241122
  0.81404956 0.82303486 0.95663655]]
{0: 292, 1: 596}
acc 0.6576576576576577
(0.49328859060402686, 0.9932432432432432, 0.6591928251121075, None)

test set
{0: 1663, 1: 2957}
acc 0.6735930735930736
(0.49949272911734865, 0.9813953488372092, 0.662034961900493, N

In [38]:
for i in np.linspace(0,1,11):
    print("alpha-init:",i)
    train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=True,penalty=2)

alpha-init: 0.0
k Tensor("Const:0", shape=(33,), dtype=float64)
s Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty2
loss Tensor("sub_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 159150.60326214734
dev set
[-0.00447464 

2 loss 146101.8036713557
dev set
[ 0.31783772  0.12210608  0.16096265  0.07221402  0.12177342  0.22570068
  0.22900776  0.14746126  0.15552667  0.09578382  0.06730813  0.16588696
  0.31022997 -0.1219712   0.31358334  0.0862623   0.2879223   0.21471383
  0.21788145  0.08852977  0.24331594 -0.08381277  0.15871157  0.14717002
  0.11021917  0.22960748  0.20777265  0.20645925  0.21663016 -0.12484925
  0.33888476  0.33928117  0.21505989]
[[0.87344591 0.82119844 0.97998965 0.73836648 0.84082826 0.91751781
  1.01236926 1.01551045 1.11005237 0.7998638  0.72228021 1.01986777
  0.74581637 1.29409924 0.80093347 0.77811016 0.65723993 0.76198245
  0.8820485  0.66482297 0.89648113 1.13800492 0.79424637 1.22812881
  0.89258996 0.99032052 0.89301669 0.88272452 0.90918854 1.20914809
  0.58456176 0.59979711 0.84069332]]
{0: 5, 1: 883}
acc 0.33896396396396394
(0.3352208380520951, 1.0, 0.5021204410517388, None)

test set
{0: 21, 1: 4599}
acc 0.3303030303030303
(0.3272450532724505, 1.0, 0.49311926605504586,

0 loss 152929.69648361602
dev set
[0.34575332 0.26982512 0.28076967 0.38056177 0.38271039 0.34848991
 0.37523786 0.27811541 0.28173941 0.38188432 0.38019838 0.2829926
 0.33669091 0.24111681 0.34068412 0.38142778 0.31237052 0.35516085
 0.36267559 0.37531022 0.34613929 0.23299208 0.37932756 0.27785258
 0.26983576 0.36560141 0.37787778 0.37927258 0.37398996 0.23953309
 0.36633358 0.36658077 0.35389949]
[[1.04167205 0.8386411  1.0253237  0.91703365 1.02198585 1.00453611
  1.10173465 1.05028927 1.16024451 0.9806308  0.9001227  1.07812181
  0.9158368  1.13302546 0.97017352 0.95831374 0.82562812 0.84551592
  0.97936118 0.75761676 1.00076672 1.00749083 0.91540728 1.18613792
  0.8993653  1.09747574 0.99287582 0.99049393 0.98896156 1.04644703
  0.75601226 0.77128675 0.91932069]]
{0: 185, 1: 703}
acc 0.4988738738738739
(0.3940256045519203, 0.9358108108108109, 0.5545545545545546, None)

test set
{0: 938, 1: 3682}
acc 0.5170995670995671
(0.40141227593699075, 0.9820598006644519, 0.5698862540967805, 

3 loss 140477.074046309
dev set
[0.75368098 0.40070815 0.44332564 0.24370833 0.26785522 0.55100069
 0.50629393 0.42562219 0.43300073 0.25731188 0.24046082 0.44873112
 0.74414125 0.06734339 0.74836057 0.25222241 0.67433319 0.5030337
 0.50280263 0.31872453 0.57867431 0.14059603 0.38984702 0.42100964
 0.3829303  0.5014517  0.46952776 0.45606274 0.49603747 0.06499691
 0.71439683 0.7185788  0.52444008]
[[0.74094756 0.85683804 1.01755597 0.70883064 0.75389851 0.8920057
  1.04166675 1.05517343 1.14359972 0.72450053 0.70704882 1.05052729
  0.61507878 1.39584403 0.66944546 0.71483253 0.61206446 0.86394279
  0.92199108 0.72214468 0.86080918 1.21825011 0.83842078 1.25014652
  0.93309829 0.9331326  0.92963632 0.92729473 0.93333347 1.31732391
  0.54872588 0.55028631 0.83405353]]
{1: 888}
acc 0.3333333333333333
(0.3333333333333333, 1.0, 0.5, None)

test set
{0: 2, 1: 4618}
acc 0.3261904761904762
(0.32589865742745777, 1.0, 0.4915890903152049, None)

4 loss 136237.21701042802
dev set
[ 0.84589949  0.4

0 loss 151070.04968550202
dev set
[0.7100312  0.57418233 0.5824313  0.68044697 0.6827378  0.62771296
 0.6535066  0.57969128 0.58607797 0.68186428 0.68005313 0.58783509
 0.69632263 0.55669071 0.70261765 0.68137777 0.63293874 0.67851152
 0.68175493 0.66942482 0.62303135 0.53824994 0.67353768 0.57802203
 0.57378252 0.68415518 0.66441633 0.66645546 0.65417283 0.54929844
 0.65369588 0.65333552 0.62843143]
[[1.01091522 0.83921447 1.02895548 0.91831226 1.02325235 1.01939494
  1.12932677 1.05355753 1.16394247 0.98189872 0.90140628 1.08156152
  0.87044758 1.11958062 0.93159832 0.95958411 0.80732299 0.83836977
  0.97683873 0.76822063 1.01802121 1.00383745 0.92632396 1.18525391
  0.89995565 1.0938882  1.0131493  1.01244895 1.01540976 1.03276681
  0.77442174 0.78369828 0.94072303]]
{0: 179, 1: 709}
acc 0.49436936936936937
(0.3921015514809591, 0.9391891891891891, 0.5532338308457712, None)

test set
{0: 948, 1: 3672}
acc 0.5175324675324675
(0.4014161220043573, 0.9794019933554817, 0.5694417616380142,

3 loss 146163.9236450166
dev set
[1.         0.62506072 0.64987757 0.53331341 0.55911585 0.81375832
 0.77469006 0.6376841  0.65779355 0.54812453 0.52973111 0.66884367
 1.         0.44141386 1.         0.54266264 0.87460684 0.51741003
 0.54685873 0.7054032  0.82623251 0.43711751 0.7479184  0.62878057
 0.61495215 0.58075862 0.76803526 0.76414151 0.76979525 0.44152815
 0.8111072  0.79971356 0.78905393]
[[0.87141684 0.9056346  1.08081944 0.62907489 0.729309   0.93024903
  1.09991771 1.11425164 1.21238929 0.68905895 0.61374113 1.12157744
  0.73765931 1.29806423 0.79509189 0.66768882 0.73357813 0.56203542
  0.68418881 0.75447004 0.91274218 1.21506976 0.89953036 1.24981215
  0.97586762 0.7995112  0.98838775 0.99436398 0.98935591 1.17801881
  0.74452309 0.75363397 0.87432198]]
{1: 888}
acc 0.3333333333333333
(0.3333333333333333, 1.0, 0.5, None)

test set
{0: 2, 1: 4618}
acc 0.3261904761904762
(0.32589865742745777, 1.0, 0.4915890903152049, None)

4 loss 143204.1860546232
dev set
[1.         0.6

0 loss 151878.4451930209
dev set
[0.98143189 0.87048724 0.87547943 0.95736725 0.96006688 0.92049363
 0.91134971 0.87361129 0.87748489 0.95906963 0.9568743  0.87930833
 0.98119868 0.87922882 0.98128172 0.95849746 0.92699809 0.95485433
 0.9589421  0.92865549 0.91622594 0.85183945 0.93582332 0.87149888
 0.87121691 0.96160889 0.9230289  0.91356621 0.90769222 0.9095905
 0.90729866 0.91719953 0.90982374]
[[1.0555966  0.84298243 1.03531526 0.94460283 1.05035843 1.02189767
  1.16147835 1.05912165 1.17150831 1.00869034 0.92755956 1.08909424
  0.91945621 1.10108257 0.97831781 0.98620248 0.832654   0.86399335
  1.00359129 0.81151755 1.02005317 0.99084238 0.96435688 1.18522444
  0.90237032 1.12151398 1.04836467 1.05879104 1.05211369 0.98686294
  0.81485175 0.82418017 0.95251222]]
{0: 342, 1: 546}
acc 0.6554054054054054
(0.4908424908424908, 0.9054054054054054, 0.6365795724465558, None)

test set
{0: 1778, 1: 2842}
acc 0.6811688311688312
(0.5056298381421535, 0.9548172757475083, 0.6611456176673568, N

3 loss 147140.4527114103
dev set
[1.48857286 0.85332978 1.48898104 1.         1.47849378 1.47838197
 1.47858942 1.4889911  1.4884422  1.4784269  0.99999997 1.48882138
 0.55264483 1.48876152 1.48894861 1.47838395 0.87366752 0.9999999
 1.47841793 0.99999999 1.47836785 0.82818621 1.         1.48328827
 0.90465087 1.47860218 1.47843266 1.47843338 1.47841405 0.91392069
 1.         1.00007363 1.        ]
[[1.62204947 0.96097956 1.51351832 1.00062428 1.6028828  1.53738981
  1.66610418 1.53548192 1.65090317 1.56095194 0.9835706  1.57100626
  0.76442603 1.58300145 1.54512995 1.53829496 0.92925157 0.9199338
  1.55581732 0.83640691 1.53137417 1.12596628 0.98985765 1.25032081
  0.97151689 1.67442583 1.56433775 1.56476272 1.55365323 1.06241122
  0.81404956 0.82303486 0.95663655]]
{0: 292, 1: 596}
acc 0.6576576576576577
(0.49328859060402686, 0.9932432432432432, 0.6591928251121075, None)

test set
{0: 1663, 1: 2957}
acc 0.6735930735930736
(0.49949272911734865, 0.9813953488372092, 0.662034961900493, N

In [41]:
for i in np.linspace(0,1,11):
    print("alpha-init:",i)
    train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=True,penalty=3)

alpha-init: 0.0
k Tensor("Const:0", shape=(33,), dtype=float64)
s Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
penalty3
loss Tensor("add_1:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 164170.59601571172
dev set
[-0.00321652 

2 loss 151651.16681535603
dev set
[ 0.31928867  0.12289352  0.16177986  0.12771741  0.30352517  0.22600436
  0.23355673  0.14813613  0.15617578  0.28046878  0.10984289  0.16673733
  0.31194872 -0.12104468  0.31517955  0.25044697  0.29026916  0.21666863
  0.22045623  0.13126633  0.24332985 -0.08367138  0.19519538  0.14705888
  0.11086409  0.23173644  0.22021416  0.22252166  0.22139604 -0.12393603
  0.33847945  0.33886846  0.21623549]
[[0.88782296 0.84249996 1.00006698 0.89893759 1.06298397 0.93382379
  1.04989737 1.03372042 1.12751414 1.04402269 0.88158914 1.04020871
  0.76282405 1.29755936 0.8167113  1.02470242 0.68841839 0.80820124
  0.92979947 0.822229   0.91017119 1.14252189 0.91245302 1.2283953
  0.91118028 1.02869167 0.96013433 0.95904619 0.94914935 1.21328318
  0.59145615 0.60640905 0.86382517]]
{0: 7, 1: 881}
acc 0.34121621621621623
(0.33598183881952326, 1.0, 0.5029736618521665, None)

test set
{0: 33, 1: 4587}
acc 0.3329004329004329
(0.3281011554392849, 1.0, 0.4940906106369008,

0 loss 157954.88424212666
dev set
[0.34793405 0.27027811 0.28130287 0.38075487 0.38282699 0.3482176
 0.3751915  0.27859417 0.28223358 0.38202589 0.38040875 0.28356297
 0.33939127 0.24141895 0.3431441  0.38158535 0.31528135 0.3550072
 0.36243603 0.37569873 0.34580439 0.23319492 0.3795161  0.27823123
 0.27027128 0.36546306 0.37797417 0.37938996 0.37398944 0.2398285
 0.36616935 0.36641326 0.35367134]
[[1.07071697 0.8471965  1.0336382  0.92915582 1.0304046  1.01203086
  1.11279271 1.05773653 1.16770936 0.99027857 0.91306168 1.08703185
  0.95079303 1.13627837 1.00228924 0.96873689 0.86435526 0.86046568
  0.99055559 0.78123572 1.0079479  1.00976431 0.92958479 1.18623042
  0.90703875 1.10530013 1.00668848 1.00284452 1.00131677 1.05048552
  0.76092083 0.77593769 0.93014747]]
{0: 187, 1: 701}
acc 0.4988738738738739
(0.39372325249643364, 0.9324324324324325, 0.5536609829488466, None)

test set
{0: 942, 1: 3678}
acc 0.517965367965368
(0.40184883088635126, 0.9820598006644519, 0.5703260659849508, No

3 loss 145465.74488052167
dev set
[0.76197243 0.40052221 0.44314605 0.4407454  0.49705443 0.5527399
 0.51952598 0.42524645 0.4326654  0.476803   0.42989391 0.44869807
 0.75447396 0.06823292 0.75773848 0.46490662 0.68390333 0.50381798
 0.50417386 0.42894073 0.57966523 0.1401767  0.4850976  0.41921311
 0.38255796 0.50335451 0.50730628 0.50255107 0.51099371 0.06588583
 0.71473392 0.71890142 0.52816769]
[[1.17192151 0.90056486 1.06120462 1.25847498 1.34524197 0.92981524
  1.14941356 1.09366593 1.18279327 1.31141777 1.24416462 1.09757085
  1.10164667 1.4030568  1.13044376 1.29297791 0.9052517  1.13820618
  1.26808289 1.0444652  0.89288357 1.22630737 1.1595409  1.25050359
  0.9705051  1.34815374 1.13427399 1.14974118 1.05206927 1.3262628
  0.60594999 0.60741438 0.89142767]]
{1: 888}
acc 0.3333333333333333
(0.3333333333333333, 1.0, 0.5, None)

test set
{0: 2, 1: 4618}
acc 0.3261904761904762
(0.32589865742745777, 1.0, 0.4915890903152049, None)

4 loss 141156.53686646651
dev set
[ 0.85676337  0

0 loss 155892.80157762434
dev set
[0.71187249 0.57309481 0.58127498 0.68739519 0.6888276  0.62868173
 0.65679645 0.57858414 0.58490061 0.6883577  0.68709939 0.58655562
 0.69818867 0.55636372 0.70447701 0.68807357 0.63019398 0.68602101
 0.68829573 0.67751373 0.62391281 0.53779874 0.68051216 0.57701638
 0.57271297 0.68954654 0.66980434 0.67161965 0.65809103 0.54899836
 0.65411432 0.65377536 0.63005555]
[[1.21381049 0.85553083 1.04443518 1.12021916 1.21389257 1.03558403
  1.18224977 1.06773639 1.17812849 1.1811273  1.10106566 1.09847909
  1.0782674  1.13143994 1.13685468 1.16281584 0.93084391 1.03274356
  1.17702764 0.93503139 1.03397613 1.00951015 1.09756197 1.18622475
  0.91509047 1.26830405 1.12818039 1.11859158 1.08328699 1.05434098
  0.80618887 0.81335689 0.9686755 ]]
{0: 181, 1: 707}
acc 0.49436936936936937
(0.3917963224893918, 0.9358108108108109, 0.5523429710867398, None)

test set
{0: 956, 1: 3664}
acc 0.5183982683982684
(0.4017467248908297, 0.9780730897009967, 0.5695492358289805,

3 loss 149961.91173308078
dev set
[1.         0.62824299 0.65330021 0.7032288  0.72835161 0.81286821
 0.77842261 0.64076731 0.66112328 0.71899313 0.6987234  0.67263288
 1.         0.44311784 1.         0.71366188 0.88815486 0.68058385
 0.71780226 0.74558598 0.82504238 0.43823453 0.77639293 0.63088594
 0.61786848 0.74294397 0.78172193 0.77754094 0.77537781 0.44329931
 0.81164307 0.80085076 0.78857143]
[[1.51460641 0.97773278 1.15087271 1.39955866 1.50650265 1.03076633
  1.38506635 1.1769911  1.27664981 1.46435524 1.38232977 1.19946259
  1.37890706 1.33912487 1.43755427 1.44161569 1.22614427 1.31807005
  1.45919876 1.22449728 1.00551293 1.2333346  1.38177222 1.25102484
  1.04078059 1.57853497 1.41307777 1.40643112 1.30637946 1.30339276
  1.06366461 1.1553916  1.02941899]]
{0: 1, 1: 887}
acc 0.3344594594594595
(0.3337091319052988, 1.0, 0.5004226542688082, None)

test set
{0: 2, 1: 4618}
acc 0.3261904761904762
(0.32589865742745777, 1.0, 0.4915890903152049, None)

4 loss 146746.2400610299
d

0 loss 156555.92246224245
dev set
[0.98275428 0.87025839 0.8753071  0.96113009 0.96320687 0.9204412
 0.91150063 0.87342296 0.87733046 0.96243159 0.96075822 0.87916424
 0.98267926 0.87910617 0.98269024 0.96199112 0.92780471 0.95925665
 0.96233315 0.93051082 0.9160167  0.85165484 0.93714601 0.87149668
 0.87101067 0.96442501 0.92357805 0.91421582 0.90782581 0.91012322
 0.90747944 0.91861847 0.90972894]
[[1.21669283 0.89636855 1.08771737 1.09978731 1.20555634 1.09328874
  1.26346012 1.10914954 1.22161135 1.16389542 1.08273135 1.14496916
  1.08151356 1.15346875 1.13995094 1.14140496 0.95238683 1.0190871
  1.1587962  0.93553188 1.09600087 1.01872921 1.09527437 1.18618982
  0.95386398 1.27666763 1.16694249 1.16697648 1.15425209 1.0890891
  0.91868604 0.93736256 1.04374574]]
{0: 342, 1: 546}
acc 0.6554054054054054
(0.4908424908424908, 0.9054054054054054, 0.6365795724465558, None)

test set
{0: 1780, 1: 2840}
acc 0.6811688311688312
(0.5056338028169014, 0.9541528239202658, 0.6609896432681243, No

3 loss 150892.6769676259
dev set
[1.4873571  0.85662825 1.48769888 1.         1.47808468 1.47794525
 1.4782045  1.48772437 1.48724226 1.47800044 1.         1.48757634
 0.65643601 1.48752353 1.48768793 1.47794763 0.9069947  1.
 1.47798926 0.99999978 1.47792844 0.81906018 1.00000578 1.48285842
 0.90074464 1.47822028 1.47800765 1.47800854 1.47798445 0.92373535
 1.00000007 1.00000105 1.        ]
[[1.53268669 1.19340765 1.42308853 1.39617013 1.51434568 1.44826647
  1.57819541 1.44506294 1.5619111  1.4720231  1.3791176  1.48099444
  1.37592795 1.49314021 1.45480991 1.44917827 1.24899261 1.31548592
  1.46684337 1.23197364 1.44220961 1.29861705 1.3917608  1.25109465
  1.25793958 1.58660268 1.47543917 1.47586797 1.46466061 1.38551317
  1.2175503  1.23393493 1.3577    ]]
{0: 292, 1: 596}
acc 0.6576576576576577
(0.49328859060402686, 0.9932432432432432, 0.6591928251121075, None)

test set
{0: 1663, 1: 2957}
acc 0.6735930735930736
(0.49949272911734865, 0.9813953488372092, 0.662034961900493, None)



In [35]:
train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=False,smooth=False)

k Tensor("Const:0", shape=(33,), dtype=float64)
s Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -15869.846624424299
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-0

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import numpy as np
def get_LF_acc(L_S,true_labels):
    #L_S : a numpy array of [NoOfDataPoints,2,NoOfLFs] 
    #true_labels : numpy array [NoOfDataPoints]
    
    tl = [-1 if x==0 else x for x in true_labels]
    unique, counts = np.unique(tl, return_counts=True)
    print(dict(zip(unique, counts)))

    # take only labels 
    L_S = L_S[:,0,:]
    #L_S shape [NoOfDataPoints,NoOfLFs]
    LF_acc = []
    for i in range(L_S.shape[1]):
#         print(accuracy_score(L_S[:,i],tl,normalize=False),accuracy_score(L_S[:,i],tl))
        LF_acc.append(accuracy_score(L_S[:,i],tl))
#         unique, counts = np.unique(L_S[:,i], return_counts=True)
#         print(i,dict(zip(unique, counts)))
#         print(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
#         LF_acc.append(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
    return np.array(LF_acc)
                      
get_LF_acc(dev_L_S,gold_labels_dev)

{1: 296, -1: 592}


array([0.02477477, 0.05292793, 0.04054054, 0.0213964 , 0.05743243,
       0.15427928, 0.00900901, 0.05292793, 0.05067568, 0.03265766,
       0.03941441, 0.04391892, 0.        , 0.02364865, 0.01013514,
       0.02702703, 0.        , 0.11599099, 0.04842342, 0.00675676,
       0.21171171, 0.33108108, 0.        , 0.01801802, 0.05405405,
       0.02477477, 0.01351351, 0.00225225, 0.00788288, 0.00112613,
       0.01238739, 0.02477477, 0.09121622])

In [61]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)


rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [64,128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

[7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2 7031.2
 7031.2 7031.2 7031.2]
batch-size: 64 alpha-init: 0.6
recall penalty


Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7fede5e7dd90>
Traceback (most recent call last):
  File "/home/vinay/snorkelEnv/lib/python3.5/_weakrefset.py", line 38, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt


0 loss 187932.14992206293
dev set
[0.59997179 0.59670596 0.59872064 0.60112702 0.60117965 0.59972785
 0.603198   0.59890187 0.60005358 0.60157675 0.60070898 0.59928264
 0.59840572 0.59944511 0.59908607 0.60195641 0.59700542 0.59961245
 0.60179755 0.59991742 0.59979421 0.59812922 0.60151936 0.60049615
 0.59736488 0.60312977 0.60222046 0.60222393 0.60208015 0.59849411
 0.59970332 0.59985586 0.60053913]
[[1.11838539 0.81561902 1.01088793 1.00048198 1.10707463 1.04250098
  1.16780285 1.03258452 1.14771374 1.06464823 0.98374433 1.06795238
  0.98151223 1.07974285 1.04185265 1.04079258 0.85431174 0.92048709
  1.05934313 0.83486947 1.03636055 0.94268387 0.99466125 1.16345007
  0.87506116 1.17652681 1.06656948 1.06698815 1.05604363 0.98963925
  0.82065053 0.83708401 0.9618773 ]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, No

4 loss 188047.18601888244
dev set
[0.69777177 0.69113718 0.69342053 0.69985774 0.6972017  0.69458953
 0.70847538 0.69344132 0.69459201 0.69988269 0.69810634 0.69395329
 0.70629094 0.69552694 0.69584186 0.70749318 0.69831657 0.69588807
 0.70206502 0.7059231  0.69538851 0.69282924 0.70757186 0.70016342
 0.69194461 0.7056146  0.70815683 0.70821337 0.70765857 0.70467514
 0.70483384 0.70446228 0.70072114]
[[1.12039494 0.82120175 1.01621812 1.0039673  1.11153187 1.04778521
  1.16387978 1.03807995 1.15322398 1.06796555 0.98767266 1.07332123
  0.97491139 1.08358032 1.04486543 1.03645659 0.84563573 0.92512669
  1.06205361 0.82895617 1.04106378 0.94800617 0.98865268 1.16353166
  0.8804976  1.17771615 1.06087546 1.06112737 1.05128374 0.98370336
  0.81701904 0.83416158 0.96349375]]
{0: 588, 1: 300}
acc 0.7207207207207207
(0.58, 0.5878378378378378, 0.5838926174496644, None)

test set
{0: 2884, 1: 1736}
acc 0.7512987012987012
(0.6025345622119815, 0.695016611295681, 0.6454797901882136, None)

batch-s

3 loss 188977.3398562435
dev set
[0.8984966  0.892551   0.89467365 0.89576517 0.89627549 0.89499413
 0.90158362 0.89476914 0.89586432 0.89656523 0.89534682 0.89520466
 0.9054111  0.89646    0.89664111 0.90019468 0.90423805 0.89428849
 0.89660643 0.90366774 0.89570009 0.89402237 0.90570379 0.90022632
 0.89327424 0.89815027 0.90463227 0.90557145 0.90177921 0.90477559
 0.89841159 0.89833921 0.89696738]
[[1.12064458 0.81987427 1.01505862 1.00507068 1.111298   1.04686618
  1.17041883 1.03682749 1.15201343 1.06899171 0.98823022 1.07215591
  0.97723733 1.08309612 1.04481779 1.04400946 0.84796591 0.92500567
  1.06393883 0.83195711 1.04014785 0.94689716 0.99098102 1.16520946
  0.87926832 1.1815255  1.0658043  1.06453819 1.05761414 0.98617758
  0.82281546 0.8390304  0.96509358]]
{0: 588, 1: 300}
acc 0.7207207207207207
(0.58, 0.5878378378378378, 0.5838926174496644, None)

test set
{0: 2889, 1: 1731}
acc 0.7519480519480519
(0.6036972848064702, 0.6943521594684385, 0.6458590852904821, None)

4 loss 

...

3 loss 188104.40615834028
dev set
[0.70078436 0.69773121 0.69968701 0.69998492 0.70068481 0.7000138
 0.70210512 0.69990156 0.70105218 0.70055124 0.6995959  0.7002563
 0.7002326  0.70038543 0.70000726 0.70083939 0.69812853 0.69890188
 0.7009422  0.69878018 0.69995858 0.69900645 0.70037858 0.70124901
 0.69832631 0.70217175 0.70109648 0.70110048 0.70098917 0.70028413
 0.69863606 0.69879866 0.69983911]
[[1.11791974 0.81462497 1.00996767 1.00099093 1.10677802 1.04174472
  1.16880217 1.03161801 1.14674569 1.06508567 0.98394356 1.06702346
  0.98196326 1.07897743 1.04120566 1.0418229  0.85292491 0.9203212
  1.05998878 0.83600062 1.0357793  0.94184285 0.99579843 1.16326382
  0.87414398 1.17746974 1.06767156 1.06809349 1.05706525 0.98992955
  0.82157852 0.83797085 0.96238157]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

3 loss 188996.2936443021
dev set
[0.90079343 0.89773203 0.89968699 0.8996065  0.900656   0.90000362
 0.9016202  0.89990194 0.90105213 0.9002473  0.89943627 0.90025614
 0.90024153 0.90038513 0.90000901 0.90034068 0.89894896 0.89879595
 0.90019067 0.89877365 0.89995006 0.89900561 0.9003769  0.90126948
 0.89832735 0.90137052 0.90107424 0.90108809 0.90087071 0.90031656
 0.89826023 0.89835084 0.89926224]
[[1.11793622 0.81462489 1.00996859 1.00102602 1.10678651 1.04174891
  1.1694926  1.03161829 1.14674621 1.06512526 0.9839721  1.06702438
  0.98196461 1.07898423 1.04121424 1.04255462 0.85292669 0.92033978
  1.06003254 0.83601639 1.03578086 0.94184438 0.99580221 1.1634053
  0.87414369 1.17788552 1.06774937 1.06811847 1.05747257 0.98993426
  0.82228564 0.83860123 0.96251996]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

3 loss 187940.36936028008
dev set
[0.60095341 0.59792068 0.59987447 0.60022906 0.6011919  0.6001943
 0.60191422 0.60009054 0.6012418  0.60086508 0.60004525 0.60044443
 0.59960218 0.60056559 0.60018669 0.60064523 0.5983111  0.59934813
 0.60081927 0.59858662 0.60014771 0.59919297 0.60018471 0.60140578
 0.59851528 0.60199677 0.60090329 0.60090747 0.60079717 0.5996815
 0.59844267 0.59860662 0.59984073]
[[1.11776472 0.8144357  1.00978041 1.00052597 1.10659235 1.04155997
  1.16898451 1.03142913 1.14655609 1.06468382 0.98368149 1.06683557
  0.98216523 1.0787994  1.04103253 1.04201339 0.85356124 0.92013057
  1.05942279 0.83619368 1.03559354 0.94165662 0.99599215 1.16323626
  0.87395535 1.17725868 1.06786349 1.06828556 1.0572541  0.99012404
  0.82176865 0.83815497 0.96198368]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None

3 loss 188533.63533087578
dev set
[0.80095798 0.79792115 0.79987491 0.79979423 0.800844   0.80019167
 0.80190952 0.80009095 0.80124202 0.8004344  0.7996227  0.80044467
 0.80004682 0.80056879 0.80018892 0.80064261 0.79875607 0.79898334
 0.80038441 0.79858613 0.80013558 0.79919303 0.80018463 0.80144005
 0.79851578 0.8015952  0.80090238 0.8009066  0.80079564 0.80012613
 0.79844331 0.79859808 0.79944168]
[[1.11776343 0.81443553 1.00978038 1.00084015 1.10659834 1.04156039
  1.16901495 1.031429   1.14655608 1.06493978 0.98378713 1.06683555
  0.98215889 1.078799   1.04103193 1.04204285 0.85311872 0.92015296
  1.05984266 0.83619487 1.03559511 0.94165671 0.99599236 1.16323286
  0.873955   1.17769286 1.06786622 1.06828741 1.05725971 0.99012108
  0.82176791 0.83820514 0.96235528]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.5910652920962199, 0.581081081081081, 0.58603066439523, None)

test set
{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, No

3 loss 189212.5385782061
dev set
[1.00140252 0.99791899 1.00032756 1.00011493 1.00129123 1.00064534
 1.00191793 1.00054532 1.00168966 1.00087691 0.99963693 1.00089568
 0.99967717 1.00101457 1.00064013 1.00065421 0.99834496 0.99897722
 1.00082634 0.99825184 1.00058647 0.99918848 1.00006552 1.0018561
 0.99851258 1.00200052 1.00091029 1.00091448 1.00080504 0.99978111
 0.99799913 0.99817356 0.99950019]
[[1.1177388  0.81443903 1.00956438 1.00062435 1.10656306 1.04139909
  1.16942024 1.03124402 1.14654301 1.0648437  0.9837678  1.06673896
  0.98248741 1.07873048 1.04087191 1.042299   0.85353161 0.92015884
  1.05973307 0.83654238 1.03542179 0.94166485 0.99619973 1.16322408
  0.87396014 1.17769329 1.06821366 1.06863663 1.05757916 0.99042951
  0.82221356 0.83858885 0.96227176]]
{0: 436, 1: 452}
acc 0.7364864864864865
(0.5685840707964602, 0.8682432432432432, 0.6871657754010695, None)

test set
{0: 2207, 1: 2413}
acc 0.7259740259740259
(0.5495234148363034, 0.881063122923588, 0.6768759571209801, No

In [11]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(2)
### smooth LFs with acc on discrete LFs
for b in [512,1024,2048]:
    for i in np.linspace(0,1,11):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

batch-size: 512 alpha-init: 0.0
precision and recall penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 179045.28391896733
dev set
[ 9.70849482e-04 -2.06232368e-03 -1.08740512e-04  2.10842547e-04
  1.26837868e-03  6.18527187e-04  1.89679405e-03  1.07709933e-04
  1.25889587e-03  8.51832761e-04  4.03058299e-05  4.61748716e-04
 -3.80751254e-04  5.82329928e-04  2.03527140e-04  6.26960851e-04
 -1.67124727e-03 -5.96042736e-04  8.00905664e-04 -1.43126302e-03
  5.58167004e-04 -7.89063528e-04  1.66734642e-04  1.42478267e-03
 -1.46670937e-03  1.97965797e-03  8.85345754e-04  8.89587708e-04
  7.79343640e-04 -3.00073892e-04 -1.57629149e-03 -1.41242930e-03
 -1.73962006e-04]
[[1.11774922 0.81441866 1.0097634  1.00041971 1.10617327 1.04113331
  1.16900118 1.03141172 1.14653858 1.0645187  0.98336604 1.06681786
  0.98258643 1.07878472 1.04101706 1.04203152 0.85354607 0.91972991
  1.05942017 0.83621146 1.03517249 0.94163819 0.9960101  1.16319961
  0.8739368  1.17727271 1.06788131 1.06830333 1.05727171 0.99054672
  0.82178795 0.83817379 0.96194619]]
{0: 266, 1: 622}
acc 0.5180180180180181
(0.39389067

3 loss 174894.90456941392
dev set
[0.10037149 0.09733871 0.09929306 0.1008106  0.10186816 0.10121919
 0.10249686 0.09950894 0.10066048 0.1014516  0.10064005 0.09986313
 0.09901977 0.09998382 0.09960412 0.10122672 0.09772911 0.10000357
 0.10140106 0.09916834 0.10115815 0.09861259 0.10076651 0.10082456
 0.09793372 0.10257985 0.101485   0.10148923 0.10137911 0.09910133
 0.09902307 0.09918694 0.10042507]
[[1.11834792 0.81501603 1.01035895 0.9998199  1.10557344 1.04053364
  1.16840093 1.03200851 1.14713431 1.06391888 0.98276624 1.06741397
  0.98318537 1.07938295 1.0416159  1.0414317  0.85414537 0.9191301
  1.05882001 0.83561187 1.03457442 0.94223556 0.99541026 1.16371437
  0.87453523 1.17667258 1.06728172 1.06770373 1.05667194 0.99114431
  0.8211887  0.83757443 0.96134832]]
{0: 263, 1: 625}
acc 0.5146396396396397
(0.392, 0.8277027027027027, 0.5320304017372421, None)

test set
{0: 1320, 1: 3300}
acc 0.5686147186147186
(0.4260606060606061, 0.93421926910299, 0.5852237252861603, None)

4 loss 1

2 loss 169042.85779520098
dev set
[0.30057201 0.2975398  0.29949454 0.30061043 0.30166799 0.30101854
 0.30229691 0.29971022 0.3008617  0.30125143 0.30043989 0.30006438
 0.29922024 0.30018475 0.2998046  0.30102656 0.29792947 0.29980344
 0.30120078 0.29896816 0.30095681 0.29881311 0.30056634 0.30102606
 0.29813528 0.30237987 0.30128473 0.30128898 0.30117891 0.29930224
 0.2988225  0.29898637 0.30022431]
[[1.11814709 0.81481445 1.01015647 1.00002007 1.10577362 1.04073635
  1.16860076 1.03180655 1.14693156 1.06411905 0.98296641 1.06721084
  0.98298474 1.07918279 1.0414152  1.04163188 0.85394492 0.91933002
  1.05902031 0.8358121  1.0347794  0.94203559 0.99561045 1.16354098
  0.87433296 1.17687278 1.06748208 1.06790406 1.05687219 0.99094382
  0.82138903 0.83777469 0.96155015]]
{0: 252, 1: 636}
acc 0.5135135135135135
(0.39308176100628933, 0.8445945945945946, 0.536480686695279, None)

test set
{0: 1316, 1: 3304}
acc 0.5642857142857143
(0.4231234866828087, 0.9289036544850499, 0.5814098565190268,

...

3 loss 168998.9529904132
dev set
[0.3009529  0.29791984 0.29987341 0.30022884 0.30128638 0.30063639
 0.30191478 0.30008973 0.30124115 0.30086983 0.3000583  0.30044383
 0.29960126 0.30056276 0.30018557 0.30064496 0.29831084 0.29942222
 0.30081894 0.29858673 0.30057704 0.299192   0.30018475 0.30140675
 0.29851445 0.30199743 0.30090343 0.30090765 0.30079738 0.29968039
 0.29844286 0.29860672 0.29984388]
[[1.11776711 0.8144365  1.00978139 1.0004017  1.10615527 1.0411155
  1.16898321 1.03142988 1.14655649 1.06450069 0.98334804 1.06683601
  0.98260436 1.07880502 1.04103497 1.04201351 0.85356388 0.91971179
  1.05940217 0.83619347 1.03515254 0.94165765 0.99599208 1.16323034
  0.87395643 1.17725492 1.06786317 1.06828522 1.05725365 0.99056691
  0.82176851 0.83815438 0.96192846]]
{0: 253, 1: 635}
acc 0.5146396396396397
(0.3937007874015748, 0.8445945945945946, 0.5370569280343717, None)

test set
{0: 1319, 1: 3301}
acc 0.5645021645021645
(0.4232050893668585, 0.9282392026578073, 0.5813566375364128, N

2 loss 165928.7302298441
dev set
[0.50100726 0.49797439 0.49992801 0.50017452 0.50123206 0.50058204
 0.50186055 0.50014432 0.50129575 0.50081551 0.50000399 0.50049842
 0.49965561 0.50061681 0.50023992 0.50059064 0.49836518 0.49936808
 0.50076457 0.4985324  0.50052284 0.4992463  0.50013043 0.50146102
 0.49856896 0.5019431  0.50084909 0.50085331 0.50074306 0.49973469
 0.49838857 0.49855243 0.49978932]
[[1.11771227 0.81438198 1.00972684 1.00045602 1.10620959 1.04117018
  1.1690374  1.03137531 1.1465019  1.06455501 0.98340236 1.06678135
  0.9825499  1.07875072 1.04098044 1.04206783 0.85350932 0.91976625
  1.05945654 0.83624781 1.03520709 0.9416034  0.9960464  1.16317698
  0.87390178 1.17730927 1.06791753 1.06833958 1.05730796 0.99051265
  0.82182276 0.83820865 0.96198334]]
{0: 279, 1: 609}
acc 0.5574324324324325
(0.42036124794745483, 0.8648648648648649, 0.5657458563535912, None)

test set
{0: 1388, 1: 3232}
acc 0.5746753246753247
(0.42883663366336633, 0.9209302325581395, 0.5851804939835339

0 loss 165066.01713229032
dev set
[0.70123416 0.69808369 0.70003716 0.70006494 0.70112248 0.70047217
 0.70175093 0.70025361 0.7014049  0.70070593 0.69989441 0.70060768
 0.69988254 0.70072684 0.70046684 0.70048106 0.69847602 0.69925804
 0.70065495 0.69842282 0.70041228 0.69935574 0.70002084 0.70157112
 0.69867877 0.70183347 0.70073947 0.7007437  0.70063346 0.69984498
 0.69827875 0.6984427  0.69967954]
[[1.11748593 0.81427296 1.00961807 1.00056561 1.10631917 1.04128005
  1.1691471  1.03126629 1.14639314 1.0646646  0.98351194 1.06667248
  0.98232314 1.07864044 1.04075377 1.04217742 0.85328472 0.91987584
  1.05956612 0.83635739 1.03531979 0.94149399 0.996156   1.16306946
  0.87379199 1.17741886 1.06802719 1.06844923 1.05741761 0.99040019
  0.82193263 0.83831861 0.96209335]]
{0: 362, 1: 526}
acc 0.6171171171171171
(0.45817490494296575, 0.8141891891891891, 0.5863746958637468, None)

test set
{0: 1860, 1: 2760}
acc 0.6400432900432901
(0.4713768115942029, 0.8644518272425249, 0.6100820633059789

4 loss 165192.78430391508
dev set
[0.80145257 0.79786731 0.7998212  0.80028301 0.80134055 0.80068696
 0.80196803 0.80003711 0.80118887 0.800924   0.80011247 0.80039151
 0.80010102 0.80050757 0.80068529 0.80069913 0.79880762 0.79947611
 0.80087302 0.79864049 0.80062879 0.79913758 0.80023892 0.80135174
 0.79846098 0.80205155 0.80095702 0.80096113 0.80085058 0.800181
 0.79849407 0.79866037 0.79988683]
[[1.11726764 0.81448939 1.00983411 1.00034755 1.1061011  1.04106657
  1.16893299 1.03148287 1.14660928 1.06444653 0.98329388 1.06688876
  0.98210483 1.07885971 1.04053547 1.04195935 0.85306688 0.91965778
  1.05934806 0.83614015 1.03510491 0.94171217 0.99593801 1.16328333
  0.87400982 1.17720079 1.06781029 1.06823253 1.05720215 0.99006887
  0.82172127 0.83810039 0.96189311]]
{0: 456, 1: 432}
acc 0.6734234234234234
(0.5069444444444444, 0.7398648648648649, 0.6016483516483517, None)

test set
{0: 2313, 1: 2307}
acc 0.7047619047619048
(0.5305591677503251, 0.8132890365448505, 0.6421825813221406, N

[1.00134575 0.99797392 1.00026616 1.00006544 1.00123426 1.00058589
 1.00186182 1.00048511 1.00163331 1.00081872 0.99997173 1.00083755
 0.99996218 1.00095687 1.00058064 1.00059482 0.9986659  0.99936348
 1.00076792 0.99848842 1.00052659 0.99924523 1.00003408 1.00179992
 0.99856599 1.00194448 1.00085223 1.00085644 1.00074652 0.99973411
 0.99834086 0.99825708 0.99978411]
[[1.11768643 0.81438335 1.00956155 1.00062431 1.10651204 1.04137566
  1.16936531 1.03122847 1.14648876 1.06480478 0.98356978 1.06669903
  0.98238076 1.07868568 1.04084879 1.04227487 0.8532289  0.91985207
  1.05969692 0.83632684 1.03540309 0.9416063  0.99616809 1.16317691
  0.87390578 1.17763821 1.0681731  1.06859588 1.05754429 0.99049012
  0.82187083 0.83850712 0.96199249]]
{0: 296, 1: 592}
acc 0.6621621621621622
(0.4966216216216216, 0.9932432432432432, 0.6621621621621621, None)

test set
{0: 1698, 1: 2922}
acc 0.6764069264069265
(0.5017111567419575, 0.9740863787375416, 0.6622995256381297, None)

3 loss 165683.50251334402


In [31]:
acc = np.array([0.02477477, 0.05292793, 0.04054054, 0.0213964 , 0.05743243,\
       0.15427928, 0.00900901, 0.05292793, 0.05067568, 0.03265766,\
       0.03941441, 0.04391892, 0.        , 0.02364865, 0.01013514,\
       0.02702703, 0.        , 0.11599099, 0.04842342, 0.00675676,\
       0.21171171, 0.33108108, 0.        , 0.01801802, 0.05405405,\
       0.02477477, 0.01351351, 0.00225225, 0.00788288, 0.00112613,\
       0.01238739, 0.02477477, 0.09121622])
### smooth LFs with acc on discrete LFs
for b in [512,1024,2048]:
    for i in np.linspace(0,1,11):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=4,debug=False)

batch-size: 512 alpha-init: 0.0
precision penalty


/home/vinay/snorkelEnv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0 loss 169509.9992925651
dev set
[ 9.70849482e-04 -2.06232368e-03 -1.08740512e-04  2.10842547e-04
  1.26837868e-03  6.18527187e-04  1.89679405e-03  1.07709933e-04
  1.25889587e-03  8.51832761e-04  4.03058299e-05  4.61748716e-04
 -3.80751254e-04  5.82329928e-04  2.03527140e-04  6.26960851e-04
 -1.67124727e-03 -5.96042736e-04  8.00905664e-04 -1.43126302e-03
  5.58167004e-04 -7.89063528e-04  1.66734642e-04  1.42478267e-03
 -1.46670937e-03  1.97965797e-03  8.85345754e-04  8.89587708e-04
  7.79343640e-04 -3.00073892e-04 -1.57629149e-03 -1.41242930e-03
 -1.73962006e-04]
[[1.11774922 0.81441866 1.0097634  1.00041971 1.10617327 1.04113331
  1.16900118 1.03141172 1.14653858 1.0645187  0.98336604 1.06681786
  0.98258643 1.07878472 1.04101706 1.04203152 0.85354607 0.91972991
  1.05942017 0.83621146 1.03517249 0.94163819 0.9960101  1.16319961
  0.8739368  1.17727271 1.06788131 1.06830333 1.05727171 0.99054672
  0.82178795 0.83817379 0.96194619]]
{0: 266, 1: 622}
acc 0.5180180180180181
(0.393890675

3 loss 165359.6199431022
dev set
[0.10037149 0.09733871 0.09929306 0.1008106  0.10186816 0.10121919
 0.10249686 0.09950894 0.10066048 0.1014516  0.10064005 0.09986313
 0.09901977 0.09998382 0.09960412 0.10122672 0.09772911 0.10000357
 0.10140106 0.09916834 0.10115815 0.09861259 0.10076651 0.10082456
 0.09793372 0.10257985 0.101485   0.10148923 0.10137911 0.09910133
 0.09902307 0.09918694 0.10042507]
[[1.11834792 0.81501603 1.01035895 0.9998199  1.10557344 1.04053364
  1.16840093 1.03200851 1.14713431 1.06391888 0.98276624 1.06741397
  0.98318537 1.07938295 1.0416159  1.0414317  0.85414537 0.9191301
  1.05882001 0.83561187 1.03457442 0.94223556 0.99541026 1.16371437
  0.87453523 1.17667258 1.06728172 1.06770373 1.05667194 0.99114431
  0.8211887  0.83757443 0.96134832]]
{0: 263, 1: 625}
acc 0.5146396396396397
(0.392, 0.8277027027027027, 0.5320304017372421, None)

test set
{0: 1320, 1: 3300}
acc 0.5686147186147186
(0.4260606060606061, 0.93421926910299, 0.5852237252861603, None)

4 loss 16

2 loss 159507.57316917385
dev set
[0.30057201 0.2975398  0.29949454 0.30061043 0.30166799 0.30101854
 0.30229691 0.29971022 0.3008617  0.30125143 0.30043989 0.30006438
 0.29922024 0.30018475 0.2998046  0.30102656 0.29792947 0.29980344
 0.30120078 0.29896816 0.30095681 0.29881311 0.30056634 0.30102606
 0.29813528 0.30237987 0.30128473 0.30128898 0.30117891 0.29930224
 0.2988225  0.29898637 0.30022431]
[[1.11814709 0.81481445 1.01015647 1.00002007 1.10577362 1.04073635
  1.16860076 1.03180655 1.14693156 1.06411905 0.98296641 1.06721084
  0.98298474 1.07918279 1.0414152  1.04163188 0.85394492 0.91933002
  1.05902031 0.8358121  1.0347794  0.94203559 0.99561045 1.16354098
  0.87433296 1.17687278 1.06748208 1.06790406 1.05687219 0.99094382
  0.82138903 0.83777469 0.96155015]]
{0: 252, 1: 636}
acc 0.5135135135135135
(0.39308176100628933, 0.8445945945945946, 0.536480686695279, None)

test set
{0: 1316, 1: 3304}
acc 0.5642857142857143
(0.4231234866828087, 0.9289036544850499, 0.5814098565190268,

...

4 loss 159399.3328776826
dev set
[0.30089871 0.29786569 0.29981932 0.30028307 0.30134061 0.3006907
 0.30196903 0.30003558 0.30118706 0.30092406 0.30011253 0.3003897
 0.29954707 0.30050835 0.30013138 0.30069919 0.29825664 0.29947653
 0.30087319 0.29864095 0.30063148 0.29913778 0.30023898 0.30135233
 0.2984601  0.30205166 0.30095766 0.30096188 0.30085161 0.29962598
 0.29849722 0.29866108 0.29989805]
[[1.11782124 0.81449055 1.00983536 1.00034747 1.10610104 1.04106122
  1.16892895 1.03148394 1.14661045 1.06444646 0.98329381 1.06689003
  0.98265851 1.07885955 1.04108912 1.04195928 0.85361804 0.91965752
  1.05934792 0.83613925 1.03509798 0.94171186 0.99593784 1.16328353
  0.87401084 1.17720069 1.06780894 1.068231   1.05719942 0.99062137
  0.82171411 0.83809998 0.96187436]]
{0: 252, 1: 636}
acc 0.5135135135135135
(0.39308176100628933, 0.8445945945945946, 0.536480686695279, None)

test set
{0: 1318, 1: 3302}
acc 0.5642857142857143
(0.4230769230769231, 0.9282392026578073, 0.5812356979405034, No

3 loss 156332.1723696476
dev set
[0.50095307 0.4979203  0.49987401 0.50022882 0.50128636 0.5006364
 0.50191491 0.50009022 0.50124176 0.50086981 0.50005829 0.50044436
 0.49960139 0.50056215 0.50018572 0.50064494 0.49831095 0.49942255
 0.50081889 0.49858669 0.50057746 0.49919202 0.50018473 0.50140644
 0.49851455 0.50199733 0.50090338 0.50090761 0.50079736 0.49968016
 0.49844306 0.49860691 0.49984345]
[[1.11776577 0.81443597 1.00978069 1.00040172 1.10615528 1.04111597
  1.16898301 1.0314293  1.14655574 1.06450071 0.98334806 1.06683524
  0.98260393 1.07880542 1.04103434 1.04201353 0.85356337 0.91971195
  1.05940224 0.83619353 1.03515241 0.94165767 0.99599209 1.16323024
  0.87395621 1.17725497 1.06786324 1.06828529 1.05725366 0.99056726
  0.82176823 0.83815412 0.96192944]]
{0: 279, 1: 609}
acc 0.5574324324324325
(0.42036124794745483, 0.8648648648648649, 0.5657458563535912, None)

test set
{0: 1383, 1: 3237}
acc 0.5735930735930735
(0.4281742354031511, 0.9209302325581395, 0.5845634753268663, 

2 loss 155468.37434595902
dev set
[0.70134397 0.69797491 0.69992861 0.70017452 0.70123206 0.70058082
 0.70186075 0.7001448  0.70129634 0.70081551 0.70000398 0.70049905
 0.69999272 0.70061629 0.70057677 0.70059064 0.69837091 0.69936762
 0.70076453 0.69853232 0.70052146 0.69924627 0.70013043 0.70146091
 0.69856931 0.70194306 0.70084896 0.70085319 0.70074297 0.69973606
 0.69838887 0.69855198 0.69978711]
[[1.11737626 0.81438167 1.00972655 1.00045603 1.10620959 1.04117269
  1.16903735 1.03137506 1.14650166 1.06455502 0.98340237 1.06678104
  0.98221345 1.07875056 1.04064409 1.04206784 0.85317956 0.91976626
  1.05945654 0.83624797 1.03521508 0.94160346 0.99604642 1.16317695
  0.87390143 1.17730928 1.0679178  1.06833984 1.05730821 0.99050277
  0.82182276 0.83820902 0.96198825]]
{0: 362, 1: 526}
acc 0.6171171171171171
(0.45817490494296575, 0.8141891891891891, 0.5863746958637468, None)

test set
{0: 1855, 1: 2765}
acc 0.6398268398268399
(0.47124773960217, 0.86578073089701, 0.610304449648712, Non

{0: 2411, 1: 2209}
acc 0.7164502164502164
(0.5441376188320507, 0.7986710963455149, 0.6472805600430802, None)

1 loss 155896.49538114632
dev set
[0.90128924 0.89802905 0.89998257 0.90012001 0.90117755 0.90052674
 0.90179882 0.90019893 0.90135028 0.900761   0.89994948 0.90055302
 0.89993761 0.90067124 0.90052192 0.90053613 0.89864609 0.89931312
 0.90071002 0.89847726 0.90046769 0.89930051 0.90007575 0.90151574
 0.89862376 0.90188855 0.90079352 0.90079731 0.90068417 0.90001704
 0.89832624 0.89849776 0.89972958]
[[1.11743085 0.81432764 1.0096727  1.00051054 1.1062641  1.04122526
  1.16910577 1.03132102 1.14644781 1.06460953 0.98345687 1.06672719
  0.98226806 1.07869617 1.04069869 1.04212235 0.85322941 0.91982077
  1.05951105 0.83630361 1.03526238 0.94154922 0.99610151 1.16312347
  0.87384702 1.17736379 1.06797457 1.06839703 1.05737198 0.99023832
  0.82189706 0.8382634  0.96204337]]
{0: 477, 1: 411}
acc 0.6835585585585585
(0.5182481751824818, 0.7195945945945946, 0.6025459688826026, None)

t

4 loss 156089.5789067435
dev set
[1.0014605  0.99786299 1.00039267 1.00017546 1.00134955 1.00070728
 1.00197494 1.00060828 1.00174702 1.00093704 1.00000836 1.00095566
 1.00000146 1.00107392 1.00070208 1.00071607 0.99873929 0.99945885
 1.0008868  0.99856399 1.00064902 0.99913589 1.00004546 1.00191318
 0.99845405 1.00205741 1.00097022 1.00097438 1.00086564 0.99961489
 0.99841626 0.99818287 0.99988796]
[[1.11779366 0.81449618 1.00956989 1.00062459 1.10661697 1.0414289
  1.16947675 1.03126551 1.14659914 1.06488781 0.98356958 1.06678386
  0.98238049 1.07877945 1.04090132 1.0423295  0.85316175 0.91979992
  1.05977476 0.83627514 1.03544677 0.94172118 0.996157   1.16328327
  0.87402081 1.17774987 1.06825916 1.0686823  1.05761972 0.99060903
  0.8217965  0.83858588 0.96190576]]
{0: 296, 1: 592}
acc 0.6621621621621622
(0.4966216216216216, 0.9932432432432432, 0.6621621621621621, None)

test set
{0: 1698, 1: 2922}
acc 0.6764069264069265
(0.5017111567419575, 0.9740863787375416, 0.6622995256381297, N

In [18]:
for b in [32,64,128,512,1024,2048]:
    print("batch-size:",b)
    train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=4)

batch-size: 32
{1: 296, -1: 592}
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
precision penalty
sft Lj Tensor("map/while/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?,), dtype=float64)
sft kj Tensor("map/while/Gather:0", shape=(), dtype=float64)
sft aj Tensor("map/while/Gather_1:0", shape=(), dtype=float64)
sft indices Tensor("map/while/Where:0", shape=(?, 1), dtype=int64)
sft l_ij_eq_kj 

0 loss 233952.1118585545
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.11911427 0.8157741  1.01111336 0.99904695 1.10481395 1.03978357
  1.16763785 1.03276307 1.14789029 1.06315656 0.98200342 1.06816907
  0.98394964 1.08014919 1.04238104 1.04066411 0.85491189 0.91839294
  1.05805315 0.83484655 1.03383244 0.94291845 0.99464479 1.16458152
  0.87527706 1.17591538 1.06651788 1.06693987 1.05590939 0.99191256
  0.8204247  0.83680628 0.96059284]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.591065292

1 loss 232542.93462455602
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.11911378 0.81577792 1.01112007 0.99905173 1.10481237 1.03977682
  1.16763825 1.03276922 1.14789484 1.06315653 0.98200339 1.06817338
  0.98394929 1.08014832 1.04238002 1.04066599 0.85491017 0.91838126
  1.05805485 0.83484754 1.03381831 0.94296761 0.99464596 1.16458067
  0.87529023 1.17591259 1.06651796 1.06694004 1.05590909 0.99191176
  0.82042425 0.83680782 0.96058797]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.59106529

2 loss 231723.23282461683
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.11814955 0.81481874 1.01016326 1.00001862 1.10577337 1.04073491
  1.16860132 1.03181185 1.14693846 1.06411857 0.98296599 1.06721788
  0.98298656 1.07918627 1.0414172  1.04163089 0.85394626 0.91933204
  1.05901951 0.83581118 1.03477191 0.94203385 0.99560979 1.16361661
  0.87433702 1.17687314 1.06748096 1.06790304 1.05687157 0.99094858
  0.82138678 0.83777232 0.96154617]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.59106529

3 loss 231622.31947743098
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.11796147 0.81463104 1.00997589 1.00020703 1.10596107 1.04092205
  1.16878922 1.03162433 1.14675099 1.06430644 0.98315389 1.06703036
  0.98279865 1.0789982  1.04122921 1.04181904 0.85375824 0.91951869
  1.0592077  0.83599916 1.0349593  0.94184985 0.99579779 1.1634284
  0.87415011 1.17706079 1.06766886 1.06809094 1.05705944 0.99076058
  0.82157464 0.83796042 0.96173372]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.591065292

4 loss 231589.04684921185
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.11782153 0.81449128 1.0098363  1.00034721 1.10610095 1.04106164
  1.16892916 1.03148463 1.14661135 1.06444636 0.98329382 1.06689076
  0.98265873 1.07885835 1.04108929 1.04195911 0.85361825 0.91965802
  1.05934778 0.83613912 1.03509906 0.94171151 0.99593777 1.16328828
  0.87401042 1.17720065 1.06780882 1.06823088 1.05719936 0.99062062
  0.82171462 0.83810044 0.96187358]]
{0: 597, 1: 291}
acc 0.7263513513513513
(0.59106529

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import numpy as np
def get_LF_acc(L_S,true_labels):
    #L_S : a numpy array of [NoOfDataPoints,2,NoOfLFs] 
    #true_labels : numpy array [NoOfDataPoints]
    
    tl = [-1 if x==0 else x for x in true_labels]
    unique, counts = np.unique(tl, return_counts=True)
    print(dict(zip(unique, counts)))

    # take only labels 
    L_S = L_S[:,0,:]
    #L_S shape [NoOfDataPoints,NoOfLFs]
    LF_acc = []
    for i in range(L_S.shape[1]):
#         print(accuracy_score(L_S[:,i],tl,normalize=False),accuracy_score(L_S[:,i],tl))
        LF_acc.append(accuracy_score(L_S[:,i],tl))
#         unique, counts = np.unique(L_S[:,i], return_counts=True)
#         print(i,dict(zip(unique, counts)))
#         print(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
#         LF_acc.append(precision_score(L_S[:,i],tl,labels=[LF_l[i]],average='macro'))
    return np.array(LF_acc)
                      
get_LF_acc(dev_L_S,gold_labels_dev)

{1: 296, -1: 592}


array([0.02477477, 0.05292793, 0.04054054, 0.0213964 , 0.05743243,
       0.15427928, 0.00900901, 0.05292793, 0.05067568, 0.03265766,
       0.03941441, 0.04391892, 0.        , 0.02364865, 0.01013514,
       0.02702703, 0.        , 0.11599099, 0.04842342, 0.00675676,
       0.21171171, 0.33108108, 0.        , 0.01801802, 0.05405405,
       0.02477477, 0.01351351, 0.00225225, 0.00788288, 0.00112613,
       0.01238739, 0.02477477, 0.09121622])

In [25]:
for b in [32,64,128,512,1024,2048]:
    print("batch-size:",b)
    train(0.001,5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=4,debug=False)

batch-size: 32
{1: 296, -1: 592}
precision penalty
0 loss 223954.06778980076
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.1591274  0.93922475 1.10151145 0.80203972 0.91245653 0.85538126
  0.96507803 1.13007062 1.23133583 0.86713989 0.78875152 1.14998827
  1.02226201 1.13294836 1.08724803 0.8400362  0.91663722 0.73398533
  0.86011235 0.63867085 0.848954   1.09974837 0.79437446 1.18623547
  0.98990485 0.9765063  0.86548673 0.86547704 0.85544074 1.03051117
  0.62578497 0.64115996 0.76714566]]


2 loss 212692.7848889567
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.03530224 0.90441376 1.0255808  0.73778983 0.84925991 0.7983581
  0.89070016 1.06213731 1.14713518 0.80062276 0.7258952  1.06453636
  0.89785153 1.01920161 0.96728965 0.76906242 0.80994052 0.67353712
  0.79285279 0.5732846  0.78844787 1.14468824 0.72374236 1.04980459
  0.9491076  0.90376529 0.793004   0.7925492  0.78340922 0.90603957
  0.56169719 0.5767034  0.6996059 ]]
{0: 493, 1: 395}
acc 0.7173423423423423
(0.5569620253

4 loss 212234.83892677928
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.09400451 0.92325361 1.0640888  0.77088991 0.88020417 0.82517113
  0.92809714 1.09703001 1.18931362 0.83369305 0.75710204 1.10751898
  0.95799662 1.07384825 1.02516044 0.80500949 0.86217081 0.70095067
  0.82701228 0.60620971 0.815538   1.13185035 0.75940573 1.11439298
  0.97219589 0.93953476 0.82937203 0.8291582  0.81946382 0.9661098
  0.59378035 0.60913699 0.73202749]]
{0: 494, 1: 394}
acc 0.7184684684684685
(0.558375634

1 loss 229922.5949763347
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.13497389 0.83163303 1.02696484 0.98316616 1.08894118 1.02391376
  1.15176759 1.04862185 1.16374346 1.04728434 0.96613316 1.0840202
  0.99980204 1.0960012  1.05823343 1.02478545 0.87076829 0.90254144
  1.04217199 0.81897618 1.01793536 0.95878545 0.97877334 1.18044891
  0.89115179 1.16004308 1.05064639 1.05106877 1.04003709 1.00776694
  0.80455076 0.82093111 0.94471404]]
{0: 584, 1: 304}
acc 0.722972972972973
(0.58223684210

3 loss 229439.5672422827
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.13592972 0.83259775 1.02793308 0.98220798 1.08797328 1.02293116
  1.15080483 1.04959066 1.16470754 1.04631782 0.96517252 1.08499074
  1.00076258 1.09695921 1.05919092 1.02382491 0.87172653 0.90155649
  1.04121033 0.81801308 1.01694998 0.95980753 0.97780992 1.18140935
  0.89213286 1.15907294 1.04968152 1.0501039  1.03907162 1.00872597
  0.80358608 0.81996654 0.94374166]]
{0: 583, 1: 305}
acc 0.7240990990990991
(0.583606557

In [28]:
for b in [32,64,128,512,1024,2048]:
    print("batch-size:",b)
    train(0.01,5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=4,debug=False)

batch-size: 32
{1: 296, -1: 592}
precision penalty
0 loss 203507.93814599
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[ 1.82498618e-03  2.11419839e-01  1.44884788e-01  2.28660711e-01
   2.75707921e-01  9.62773149e-02  1.90616828e-01  2.12745959e-01
   2.26040021e-01  2.58129272e-01  2.30839441e-01  1.52956122e-01
  -1.04826231e-01  4.74937588e-02 -2.50049457e-02  1.45888940e-01
  -1.17732440e-01  1.40559253e-01  9.04861430e-02  1.56463876e-01
  -1.21465206e-01  2.23317165e+00  1.75514471e-01 

2 loss 190660.48340389767
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[-1.32680924e-01  1.01512489e-01  3.13192868e-02  2.10590822e-01
   1.61255421e-01 -1.38268967e-01  2.00387925e-01  9.74874430e-02
   1.02613576e-01  1.98206192e-01  1.76633975e-01  3.28918179e-02
  -2.33192998e-01 -9.28838172e-02 -1.60091162e-01  1.68955758e-01
  -2.32593950e-01  2.99547953e-02  6.53426461e-02  2.29326661e-01
  -5.36470672e-01  3.15071568e+00  2.30944830e-01 -1.67176200e-01
   1.70388356e-01  1.63530784e-0

4 loss 189351.8590686145
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[-0.10287645  0.12707793  0.06012584  0.21220079  0.18915768 -0.07609392
   0.19368088  0.12296443  0.12599935  0.2165935   0.18652282  0.05752074
  -0.20269695 -0.07691877 -0.13644151  0.16213866 -0.20741225  0.05778901
   0.07643242  0.21110687 -0.4186846   2.97864986  0.21394866 -0.13451057
   0.19129202  0.17780732  0.21084824  0.20913115  0.18265098 -0.18849408
   0.17860582  0.19842494  0.00565208]]
{0: 230, 1: 658}
ac

1 loss 216975.6625429358
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.26093936 0.97135884 1.16165186 0.83355968 0.93977223 0.87549596
  1.00145467 1.18448993 1.29736633 0.89774169 0.81694974 1.21734302
  1.12539758 1.224285   1.18505411 0.87467441 1.00061283 0.75429601
  0.89236498 0.66951389 0.86903307 1.10496951 0.82878478 1.30282369
  1.03004807 1.01000361 0.90052503 0.90090036 0.88996443 1.13344685
  0.65521926 0.67148466 0.79519372]]
{0: 494, 1: 394}
acc 0.7184684684684685
(0.558375634

3 loss 215146.16865887708
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.25899292 0.97429331 1.16276026 0.82685289 0.93302896 0.86894255
  0.99431343 1.18600311 1.29802155 0.89086173 0.81028063 1.21800676
  1.12354638 1.2231532  1.18350806 0.86769769 0.99994092 0.74752651
  0.88554241 0.66270354 0.86235068 1.11112397 0.82176958 1.30005298
  1.03282573 1.00287138 0.89343324 0.89378615 0.88288869 1.13157924
  0.64845755 0.66469777 0.78827156]]
{0: 494, 1: 394}
acc 0.7184684684684685
(0.55837563

In [19]:
for b in [32,64,128,512,1024,2048]:
    print("batch-size:",b)
    train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                LF_acc = get_LF_acc(dev_L_S,gold_labels_dev) ,pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False,penalty=5)

batch-size: 32
{1: 296, -1: 592}
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 228354.28576073676
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e

2 loss 228017.41723589378
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.12222546 0.81888496 1.01421667 0.99591543 1.10169461 1.03667123
  1.1645088  1.03586846 1.15099131 1.06003201 0.97887948 1.07127469
  0.98707583 1.0832638  1.04550215 1.03753245 0.85803951 0.91528312
  1.05492743 0.83171602 1.0307355  0.94595551 0.9915135  1.1677022
  0.87837856 1.17278967 1.06338736 1.06380936 1.05278043 0.99503895
  0.81729574 0.83367699 0.95747771]]
{0: 596, 1: 292}
acc 0.7274774774774775
(0.592465753

marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 228562.58495258252
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04 -1.64293569e-04  6.66140471e-04
 -1.76181555e-04  7.85815884e-04  4.08124727e-04  4.22361165e-04
 -1.46658659e-03 -8.00654606e-04  5.96248234e-04 -1.63583977e-03
  3.53762393e-04 -5.85646273e-04 -3.78531600e-05  1.63011132e-03
 -1.26266566e-03  1.77477556e-03  6.80857197e-04  6.85077792e-04
  5.74762734e-04 -9.65627854e-05 -1.78008441e-03 -1.61622533e-03
 -3.78494646e-04]
[[1.11774948 0.81441897 1.00976392 1.0004194  1.1061734  1.04113457
  1.16900149 1.03141216 1.14653903 1.06451874 0.98336602 1.06681834
  0.98258638 1.07878618 1.04101704 1.04203139 0.85354598 0.91973095
  1.05942006 0.83621143 1.035172   0.94163735 0.99601009 1.16321588
  0.87393734 1.1772732  1.06788121 1.06830326 1.05727174 0.99054831
  0.82178707 0.8381727

{0: 2972, 1: 1648}
acc 0.7482683982683983
(0.6037621359223301, 0.6611295681063123, 0.6311449413257215, None)

batch-size: 2048
{1: 296, -1: 592}
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 33), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(33,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(33,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 228573.0921039395
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05

In [77]:
import scipy.sparse as sp
import _pickle as pkl
# L_train = pkl.load(open("train_L_S_discrete.p","rb"))
# L_train = sp.csr_matrix(L_train)

# L_gold = pkl.load(open("gold_discrete.p","rb"))
# print(np.array(L_gold).shape)
# L_gold = sp.csr_matrix(L_gold)

L_train = np.load("train_L_S_discrete.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)


from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train,  cardinality=2)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))

(8272, 33)
started at: 13-6-2018, 22:20:29
trained in  0:00:28.347146


In [78]:

L_dev = np.load("dev_L_S_discrete.npy")
L_dev = L_dev[:,0,:].astype(int)
print(np.array(L_dev).shape)
L_dev = sp.csr_matrix(L_dev)

import numpy as np
marginals = gen_model.marginals(L_dev)
marginals = np.array(marginals)
print(marginals.shape)
# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_dev),GenLabels,average="binary"))


(888, 33)
(888,)
(888,)
(0.5381679389312977, 0.9527027027027027, 0.6878048780487805, None)


In [80]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)

%time L_train = labeler.apply(split=0)

gen_model = GenerativeModel(lf_propensity=True)
gen_model.train(L_train, decay=0.95, step_size=0.1/L_train.shape[0], reg_param=0.0
)

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
MentionScorer(dev_cands, L_gold_dev).score(marginals)

Clearing existing...
Running UDF...
[=                                       ] 0%

ValueError: 
                        Unable to parse label with value (0, 0)
                        for candidate with values [True, False]

Inferred cardinality: 2
Scores (Un-adjusted)
Pos. class accuracy: 0.953
Neg. class accuracy: 0.591
Precision            0.538
Recall               0.953
F1                   0.688
----------------------------------------
TP: 282 | FP: 242 | TN: 350 | FN: 14



({ChemicalDisease(Span("b'Amiodarone'", sentence=10440, chars=[0,9], words=[0,0]), Span("b'pneumonitis'", sentence=10440, chars=[122,132], words=[18,18])),
  ChemicalDisease(Span("b'BHA'", sentence=6134, chars=[19,21], words=[5,5]), Span("b'forestomach tumors'", sentence=6134, chars=[211,228], words=[37,38])),
  ChemicalDisease(Span("b'BHA'", sentence=6138, chars=[64,66], words=[11,11]), Span("b'forestomach carcinogenesis'", sentence=6138, chars=[68,93], words=[12,13])),
  ChemicalDisease(Span("b'BHA)'", sentence=6132, chars=[84,87], words=[14,15]), Span("b'forestomach tumorigenesis'", sentence=6132, chars=[101,125], words=[19,20])),
  ChemicalDisease(Span("b'Cefotetan-induced'", sentence=9000, chars=[0,16], words=[0,0]), Span("b'hemolytic anemia'", sentence=9000, chars=[25,40], words=[2,3])),
  ChemicalDisease(Span("b'Cocaine-induced'", sentence=7230, chars=[0,14], words=[0,0]), Span("b'seizures'", sentence=7230, chars=[26,33], words=[2,2])),
  ChemicalDisease(Span("b'Cyclophosphamide

In [ ]:
L_dev = labeler.apply_existing(split=1)
_ = gen_model.error_analysis(session, L_dev, L_gold_dev)

In [74]:

L_test = np.load("test_L_S_discrete.npy")
L_test = L_test[:,0,:].astype(int)
print(np.array(L_test).shape)
L_test = sp.csr_matrix(L_test)

import numpy as np
marginals = gen_model.marginals(L_test)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_test),GenLabels,average="binary"))


(4620, 33)
(4620,)
(4620,)
(0.5415730337078651, 0.9607973421926911, 0.6926946107784431, None)


In [46]:
#majority vote #84 # Regex
# train_L_S_df.mode(axis=1)[0].tolist()
import math
# test_L_S_df.mode(axis=1)[0].tolist()
test_L_S_df = pd.DataFrame((np.array(test_L_S)[:,0,:]))
# test_L_S_df = test_L_S_df.astype(int)
test_L_S_df = test_L_S_df.replace(0, np.NaN)

predicted_labels=test_L_S_df.mode(axis=1)[0].tolist()

predicted_labels = [ int(x) if not math.isnan(x) else 1 for x in predicted_labels ]
predicted_labels = [ 0 if x == -1 else 1 for x in predicted_labels ]
# print(predicted_labels)
# Results = predictAndPrint(predicted_labels)
print("acc",accuracy_score(gold_labels_test,predicted_labels))
print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(predicted_labels),average="binary"))
           

acc 0.6409090909090909
(0.46290944123314065, 0.6385382059800665, 0.5367215861491204, None)


In [45]:
#majority vote #84 # Regex
# train_L_S_df.mode(axis=1)[0].tolist()
import math
# test_L_S_df.mode(axis=1)[0].tolist()
test_L_S_df = pd.DataFrame((np.array(test_L_S)[:,0,:]))
# test_L_S_df = test_L_S_df.astype(int)
test_L_S_df = test_L_S_df.replace(0, np.NaN)

predicted_labels=test_L_S_df.mode(axis=1)[0].tolist()

predicted_labels = [ int(x) if not math.isnan(x) else -1 for x in predicted_labels ]
predicted_labels = [ 0 if x == -1 else 1 for x in predicted_labels ]
# print(predicted_labels)
# Results = predictAndPrint(predicted_labels)
print("acc",accuracy_score(gold_labels_test,predicted_labels))
print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(predicted_labels),average="binary"))
           

acc 0.7482683982683983
(0.6107512953367875, 0.626578073089701, 0.618563463430633, None)
